<a href="https://colab.research.google.com/github/zuzannazak/SentimentAnalysisInPolish/blob/main/Sentiment_analysis_in_Polish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Combining multiple files
**Input:** Csv files relating to 1 profile gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i, uploaded to Colab in folder called DataToCombine

**Output:** One CSV file

In [ ]:
import os
import glob
import pandas as pd
os.chdir("/content/DataToCombine")

In [ ]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [ ]:
#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

#Sentiment analysis
**Input:** Prepared csv files gathered from https://colab.research.google.com/drive/1P0mf1jhhYLqlDVN2eqC2HjbWeEg-6dmJ#scrollTo=QhTw_07h092i and combined per social media account

**Output:** sentiment assigned to each tweet

In [1]:
pip install -i https://pypi.clarin-pl.eu/ lpmn_client==1.4.1

Looking in indexes: https://pypi.clarin-pl.eu/


In [2]:
 pip show lpmn_client

Name: lpmn-client
Version: 1.4.1
Summary: Client for writing LPMN queries.
Home-page: UNKNOWN
Author: Szymon Ciombor, Mateusz Gniewkowski
Author-email: Szymon.Ciombor@pwr.edu.pl, mateusz.gniewkowski@pwr.edu.pl
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: pyyaml, requests
Required-by: 


#Sentiment analysis from csv file


In [12]:
import pandas as pd
import numpy as np

import os

from lpmn_client import download_file, upload_file
from lpmn_client import Task

In [4]:
df = pd.read_csv("/content/SearchTweetPaczkomatyPL('2022-01-12T07_05_13.009Z',).csv")
df['sentiment'] = None
df = df.drop(columns=['Unnamed: 0'])

In [5]:
df.head()

,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,sentiment
0,Twitter for Android,1475050133461032963,"@PaczkomatyPL Pewnie, że tak 😊 Już wszystko za...",2021-12-26T10:40:33.000Z,pl,2.565559e+09,everyone,52522222,1475053917746696196,"[{'type': 'replied_to', 'id': '147505300392113...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
1,Twitter for Android,1475050133461032963,"Pierwszy raz mam taką sytuację, że nie mogę od...",2021-12-26T10:25:31.000Z,pl,NaN,everyone,52522222,1475050133461032963,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
2,Twitter Web App,1474893391817232390,@PaczkomatyPL \nJest jakiś problem po waszej s...,2021-12-26T00:02:41.000Z,pl,2.565559e+09,everyone,2799071567,1474893391817232390,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
3,Twitter for Android,1474623367227617285,@CicheMale @MeteoprognozaPL @ebonit_7 @Doppelm...,2021-12-25T22:04:57.000Z,pl,7.113024e+17,everyone,1376873115091881986,1474863762792493059,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
4,Twitter Web App,1474623367227617285,@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenB...,2021-12-25T11:42:00.000Z,pl,7.113024e+17,everyone,2651114600,1474706994225463301,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None


In [ ]:
for i, row in df.iterrows():
    unique_id = i
    val=(df['text'].values[i])
    with open('plik.txt', 'w') as f:
      f.write(val)
    file_id = upload_file("/content/plik.txt")
    t = Task('any2txt|wcrft2|wsd|ccl_emo({"lang":"polish"})|ccl_emo_stats({"lang":"polish", "split_paragraphs": false}) ')
    output_file_id = t.run(file_id)
    downloaded = download_file(output_file_id, "./out")

    try:
      df_sentiment=pd.read_csv(downloaded, compression='zip', header=0, sep=';')
      df_sentiment = df_sentiment.replace("None", 0)
      df_sentiment = df_sentiment.replace("amb", 0)
      df_sentiment["Polarity"] = pd.to_numeric(df_sentiment["Polarity"])
      column_sum = df_sentiment["Polarity"].sum()
      df['sentiment'].values[i]= column_sum
      print(df['text'].values[i] + ": " + str(column_sum))
    except:
      df['sentiment'].values[i]= None
      print(df['text'].values[i] + ": no sentiment for value " + str(i) )

df.to_csv('Paczkomatysentyment.csv')


In [9]:
df_sentiment

,Lemma,Syn_id,Polarity,Emotion_names,Emotion_valuations,Units in synset
0,korzystać,1377,0,NaN,NaN,czerpać_korzyści.1(38:cpos) dobrze_wychodzić.1...
1,konkurencja,1340,0,NaN,NaN,kompetycja.1(8:zdarz) wyścig.2(8:zdarz) współz...


In [8]:
df

,source,conversation_id,text,created_at,lang,in_reply_to_user_id,reply_settings,author_id,id,referenced_tweets,public_metrics,geo,sentiment
0,Twitter for Android,1475050133461032963,"@PaczkomatyPL Pewnie, że tak 😊 Już wszystko za...",2021-12-26T10:40:33.000Z,pl,2.565559e+09,everyone,52522222,1475053917746696196,"[{'type': 'replied_to', 'id': '147505300392113...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,-2
1,Twitter for Android,1475050133461032963,"Pierwszy raz mam taką sytuację, że nie mogę od...",2021-12-26T10:25:31.000Z,pl,NaN,everyone,52522222,1475050133461032963,NaN,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
2,Twitter Web App,1474893391817232390,@PaczkomatyPL \nJest jakiś problem po waszej s...,2021-12-26T00:02:41.000Z,pl,2.565559e+09,everyone,2799071567,1474893391817232390,NaN,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
3,Twitter for Android,1474623367227617285,@CicheMale @MeteoprognozaPL @ebonit_7 @Doppelm...,2021-12-25T22:04:57.000Z,pl,7.113024e+17,everyone,1376873115091881986,1474863762792493059,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
4,Twitter Web App,1474623367227617285,@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenB...,2021-12-25T11:42:00.000Z,pl,7.113024e+17,everyone,2651114600,1474706994225463301,"[{'type': 'replied_to', 'id': '147462336722761...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
936,Twitter for Android,1479235375117778946,@OLogistyczny @InPostPL1 Chyba znowu oznaczyli...,2022-01-06T23:43:47.000Z,pl,1.382288e+18,everyone,1227343711278944257,1479237288097943555,"[{'type': 'replied_to', 'id': '147923537511777...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
937,Twitter for Android,1478035206904950788,@PaczkomatyPL @RBrzoska Czyli mam czekać 30 dn...,2022-01-06T18:23:59.000Z,pl,8.590470e+17,everyone,859047002381967361,1479156810233700355,"[{'type': 'replied_to', 'id': '147908802222645...","{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN,None
938,Twitter for iPad,1478833901510475785,@PaczkomatyPL @HERYTlERA A czy w piątek kurier...,2022-01-06T16:24:41.000Z,pl,2.565559e+09,everyone,291712315,1479126786923548672,"[{'type': 'replied_to', 'id': '147883390151047...","{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN,None
939,Twitter Web App,1478035206904950788,@PaczkomatyPL 2/2 do tego reklamacja przez str...,2022-01-06T13:50:39.000Z,pl,8.590470e+17,everyone,859047002381967361,1479088022226452480,"[{'type': 'replied_to', 'id': '147908719495022...","{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN,None


#Visualisation
Materials:

https://medium.com/@siddhardhan23/data-visualization-in-python-a90ddb706b23

https://towardsdatascience.com/5-python-gui-frameworks-to-create-desktop-web-and-even-mobile-apps-c25f1bcfb561



@PaczkomatyPL Pewnie, że tak 😊 Już wszystko załatwione z Waszym konsultantem! Dogadać się można zawsze i szybko ❤: -1
Pierwszy raz mam taką sytuację, że nie mogę odebrać paczki z @PaczkomatyPL ,bo paczkomat ... zamarzł 😆❄🌨 I nie otwiera się 😭😭: 0
@PaczkomatyPL 
Jest jakiś problem po waszej stronie?
Nie można doładować konta.
Otrzymuję komunikat:
Upsss
Wystąpił nieoczekiwany błąd. Proszę spróbować później.: 0
@CicheMale @MeteoprognozaPL @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Wzajemnie...Spokojnych Świąt 🌲🌟 https://t.co/vMYFv050EN: 0
@CicheMale @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Wesołych świąt! Bo śniegu chyba nie brakuje:): -1
RT @PrezesObywatel: @MariuszMorda @BartoszKowara U mnie tak samo. @PaczkomatyPL robią dobra robotę. Wszystko w ciągu 1 czy 2 dni.: 0
@MariuszMorda @BartoszKowara U mnie tak samo. @PaczkomatyPL robią dobra robotę. Wszystko w ciągu 1 czy 2 dni.: 0
@Favoreq @PaczkomatyPL Ale dziewczyny maja dzięki Tobie @Favoreq cudowne święta! Dziś Pani Ela idzie do znajomych, jutro z Tobą ♥️ namawiałam ją aby przyszła do nas na wigilie, ale się nie zgodziła. Tatiana, cudownie, ze jesteś ♥️♥️♥️♥️: 0
@OkunAdam @AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Nie korzystam z konkurencji https://t.co/HaHjmyY5RK: 0
Nikt nie zrobił więcej dla zdrowia publicznego w czasie pandemii niż @PaczkomatyPL . Szczególnie w czasie świąt.

Fakt nie opinia.: 0
@PaczkomatyPL @MariuszMorda Teraz jeszcze dopracować żeby dochodziły całe i będzie perfecto 👌: 0
@dhl_kurier @AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Nie stać ich... 😉: 0
@MariuszMorda @PaczkomatyPL in post już 2-3 lata temu zostawił konkurencję daleko z tyłu... Mają najlepiej ustawioną logistykę i są tego efekty, dlatego każdy prezent miał na święta 🎅 🎄 @RBrzoska jest Mistrzem!!! 💪👌: 2
@dhl_kurier @AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Odbierałbyś w paczkomacie awizo...: 0
Prezenty nie muszą być pięknie opakowane, żeby były radosne! W tej paczce, którą całą minutę odrywałam od @PaczkomatyPL na mrozie, bo jakimś cudem przymarzła, jest osiem kilo puszek dla Zuzi pani Eli 🐕🎅 Jutro zawiozę 🤗 https://t.co/IIcwuV1McO: 1
@CicheMale @MeteoprognozaPL @ebonit_7 @DoppelmayrGmbH @_PistenBully_ @PaczkomatyPL @bp_Polska @festiwalgorski @ewabilanstoch @PhilippLehmkuhl @OnyxBoox Dziękujemy i dla Was wszystkiego co najlepsze... https://t.co/FtDGbKQxjC: 1
@AdahoPi @n4k1r @PaczkomatyPL Nie naliczy. A przynajmniej mi kiedyś nie naliczyło, jak paczkę, której deadline odbioru upływał w niedzielę o 11 odebrałem w niedzielę o 17.

Ale nie zdziwiłbym się, jakby opróżniali jutro, więc raczej bym się wybrał przed tą 10.

A jak paczka ważna to można ją przedłużyć.: 0
Zgadzam się. @PaczkomatyPL jestem pełen podziwu 👏👏👏 https://t.co/FiUYnJ1kDq: 1
@n4k1r @PaczkomatyPL Olej, ja mam poprawiony czas do 26 😉: 2
@n4k1r @PaczkomatyPL Raczej nie będą opróżniać, ale nie wiem czy i tak ci nie policzą dodatkowej opłaty za opóźnienie i nie otworzy jak jej nie opłacisz.: 1
Ej, jest sprawa. Właśnie mi @PaczkomatyPL przypomnieli w apce, że do jutra do 10 mam odebrać paczkę.

Kurierzy będą jutro opróżniać paczkomaty? Mam się zbierać o 9 i drałować do miasta, czy olać i spokojnie odbiorę po 18?: 1
@dhl_kurier @MariuszMorda @PocztaPolska @PaczkomatyPL Pamiętam.: 0
@AdamDee_ASD @MariuszMorda @PocztaPolska @PaczkomatyPL Poczta miała zamiar wykupić Inpost z całą infrastrukturą paczkomatowa: 0
@PaczkomatyPL Tak! Po Świętach! Gratuluję dobrego samopoczucia. Gdybym wiedział, że tyle czasu potrzebujecie Państwo na dostarczenie paczki z Łodzi do Piotrkowa (40 km) pominąłbym Państwa w łańcuchu dostaw. Dzięki za lekcję!: 0
@MariuszMorda @PocztaPolska powinna zakupić @PaczkomatyPL i zrobić z tym porządek. W głębokim duchu logiki #CieciaRP ☝️😶: -1
@MariuszMorda @PaczkomatyPL  dejcie Panu zniżeczke: 0
@ElzbietaZiober @PaczkomatyPL Nie ma szans. Przy tej pogodzie to jeden dzień jazdy i drugi na powrót.: 2
@PawelBednarek1 @PaczkomatyPL Samo życie, bardzo smutne będą.
W zeszłym roku nie przyjechały, bo nie bylismy jeszcze zaszczepieni, w tym roku miały być święta full-wypas i masz... zapalenie oskrzeli dopadło najmłodszą.
Wpadaj, odległość nie ma znaczenia - @PaczkomatyPL dowodem :): -2
@KarenMillen @PaczkomatyPL Thank you John, I have sent you a DM, just awaiting your reply.: 0
@zawiszowski @PaczkomatyPL To akurat prawda mi też tak przyszedł: 0
@ElzbietaZiober @PaczkomatyPL Smuteczek. Sami zostaliście. Chętnie bym wpadł ale to drugi koniec Polski.: 2
@ktosullivan_uk @PaczkomatyPL Hello there,

I'm glad you told me about this issue. Sorry about this. Please can you drop us a private message and  I'll get this looked into for you. -john: 0
.@PaczkomatyPL jesteście WIELCY!❤️
Wczoraj nadane paczki, dzisiaj dostarczone (ok.400km)
Wnusia zachorowała,nie przyjadą na święta.
Płacz,rozpacz,wszystkie prezenty u nas, dramat...
Liczyłam na cud, cudu nie było - była fantastyczna praca Waszych pracowników. Bardzo dziękuję😍: 1
Still waiting for my delivery which should have been here Monday. The worst courier system in the world InPost are predicting delivery for 23rd December. That would be a Christmas miracle! Appalling service. @KarenMillen @PaczkomatyPL: -2
@zawiszowski @PaczkomatyPL Mi dziś przyszło PS5 ale od MediaMarkt i kuriera DPD :): 0
🎅 W Wigilię uczcijmy gromkim "Alleluja!" wszystkich kurierów oraz menadżerów logistyki InPostu. Wczoraj ostatni prezent, który byłem przekonany że już nie dojdzie, znalazł się w moim paczkomacie... 9h po wysłaniu przez sklep!

@PaczkomatyPL - robicie to zajebiście dobrze 💛 https://t.co/xDUAqBKrTA: 1
RT @PaczkomatyPL: Pierwszy w Polsce system wtórnego obiegu opakowań w e-commerce staje się faktem! Wspólnie z marką @Modivo testowo wdrażam…: 1
@mrsGasky @PaczkomatyPL Paczkomaty zdecydowanie rządzą jeśli chodzi o dostawy 💛: -1
@mrsGasky @PaczkomatyPL Inpost ogarnia jak złoto ✨: 0
Serio, @PaczkomatyPL w tym roku taki kox. Ja widzę paczkomat z okna i dosłownie co chwilę chłopaki podjedzaja. Nawet o chorych godzinach w nocy. 🥰❤️ https://t.co/yymsTHBLpg: 0
@PaczkomatyPL @RBrzoska podobno 98% paczek dociera w 24h. Ta, na którą czekam, nie została nawet odebrana z paczkomatu przez kuriera w takim czasie. Dzięki Paczkomaty! https://t.co/kaClUrmuS1: 0
RT @PaczkomatyPL: Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszyst…: 1
Halo @PaczkomatyPL proszę się ogarnąć i oddać paczkę mojej siostrze tak się nie robi wy dranie: -1
Dzięki @PaczkomatyPL ciężki okres świąteczny za nami. Wszystkiego dobrego dla wszystkich 👌 https://t.co/lRAULBvphu: -1
RT @PaczkomatyPL: Już tu jest! Paczkomat #16000 wyposażony w czujnik powietrza stanął w Gorzowie Wielkopolskim - mieście partnerskim progra…: 0
📦 @PaczkomatyPL płacą złotówkę za polubienie 💻 👍
https://t.co/W33M5FRHx5: 4
@PaczkomatyPL Zobaczcie swoje DM bo już zgłaszałem wam o dwóch paczkach. Kolejna paczka niby dwa razy mnie nie było w domu a kurier przez telefon powiedział, że mam się walić. Amazon mi chciał to wynagrodzić, że paczka nie dojdzie i wysłał nową i wyglada jak na zdjęciach.: 0
@PaczkomatyPL Medal natomiast nalezy sie Pani z punktu odbioru, ktora proaktywnie pracowala nad rozwiazaniem, gdy jeszcze panowie z infolinii betradnie rozkladali rece👍👍: 0
@PaczkomatyPL Wy cały to samo. 4 paczki i z 4 paczkami z tym samym kurierem problem. Ogarnijcie swoich pracowników: 0
@PaczkomatyPL Dziękuję!!!🎄❤🎄❤🎄❤Nawzajem!!! Podziwiam Waszą pracę i dziękuje za nią!🎄❤🎄❤🎄❤: 0
@miodny @AmazonHelp @PaczkomatyPL Zwierzęta ? to bezmózgie stworzenia.: -1
@Krzysiec @AmazonHelp @PaczkomatyPL Ręce opadają. Jestem na bank pewny, że znowu kurier na złość bo miał mi do domu wysłać bo miał już w doręczeniu a nie chciało mi się z nim ścierać dzisiaj wiec zmieniłem na paczkomat a tu taki zonk. Wrócę to sprawdzę co w środku jest: -2
@PaczkomatyPL Dokładnie przed chwilą dostałam informację,że przesyłka jest magazynowana w paczkomacie tymczasowym 😭 
Dziękuję za odzew 😁🙏: 1
./out/a0427230-3add-4158-828c-a78e61c13c88.zip is empty! No sentiment for value 53
#DPD #DHL  #POCZTEX @PaczkomatyPL  #PocztaPolska  🎄❤🎄❤🎄❤ https://t.co/7wN9QOKLK4: no sentiment
@PaczkomatyPL Sezam otwarty, o 16.30 możliwy odbiór z innego paczkomatu, paczki w domu.🎁 Zakładam, że nastąpiła zwykła koincydencja 😉. Szkoda tylko że ci konsultanci na infolinii tacy nieskuteczni. Wesołych Świąt! 🎄: -1
./out/e3308d29-d156-496f-b932-78579f3dfda4.zip is empty! No sentiment for value 55
@PaczkomatyPL 🧐: no sentiment
@norweskidrwal @PaczkomatyPL A generalnie transit time z D0 na D1 w obrębie Polski na rynku większych graczy w kurierce to standard :-) ale rzeczywiście paczkomaty są meeega wygodne i średnio co 2-3 klient wybiera ta metodę doręczenia w przypadku paczek małogabarytowych :-): -2
@norweskidrwal @PaczkomatyPL W takich okresach jak teraz dostawy w sobotę są dość popularne w sieciach kurierskich. Z reguły w E-commerce największy peak jest w poniedziałek, po weekendzie. Gdy dochodzą święta, Black Friday itd kurierzy sobotami rozładowują wolumen, żeby realizować wymagane KPI :-): 0
@LeszekJarema @AmazonHelp @PaczkomatyPL W jakiej Ty dziczy mieszkasz, że takie zwierzęta tam paczki wożą? 😡: 0
@LeszekJarema @AmazonHelp @PaczkomatyPL O ciąg dalszy podglądania co w paczuszce siedzi na święta!! To samo było u mnie kiedyś. 🤷‍♂️🤦‍♂️🤦‍♂️🤦‍♂️: -2
@Dixon_TSW @norweskidrwal @PaczkomatyPL Do mnie dzisiaj zajechał Pocztex. Wbił pod ganek, niemal ze driftem otworzył okno i rzucił paczką przed drzwi. Były tam jakieś ubraniowe rzeczy, wiec luz, ale wyglądało to jakby bombę podrzucił xD: -3
@norweskidrwal @PaczkomatyPL To dosyć częste przy Paczkomatach 👍🙂 Mnie ostatnio zaskoczyło DPD. Zamówienie w piątek o 13, w sobotę o 8:45 kurier w drzwiach 😅: 0
@forsalpl @MBalawelder Brzoska @PaczkomatyPL  chyba umrze ze śmiechu. Ja chyba też. Ile tam mają związków zawodowych, ostatnio było 100 ? https://t.co/VGZwUFZwQp: 0
@masha8423 @PaczkomatyPL А, tak! Trzeba było dać #! Masz rację!: 0
./out/d0e634b4-601f-40c5-8eb3-56a1fbbc640e.zip is empty! No sentiment for value 64
@PaczkomatyPL 👏👏👏: no sentiment
./out/edc7d76f-e275-42c9-80e0-327b09f60012.zip is empty! No sentiment for value 65
@AmazonHelp @PaczkomatyPL https://t.co/m08NyWXsYs: no sentiment
@AmazonHelp Właśnie odebrałem paczkę ale nie wiem czy coś jest w środku. Paczka rozwalona. Miała być dostarczona do domu ale przekierowałem na paczkomat bo dziś mnie nie było a w środku to. Po numerze telefonu z aplikacji widzę, że znowu ten sam kurier @PaczkomatyPL co z tym mam zrobić ? https://t.co/g489VmVgNL: -1
@Stalowy_Szczur @PaczkomatyPL buziaka ode mnie: 0
@Helciunia To ja się dołączam! @PaczkomatyPL , #DHL, #DPD: 0
@NhTs1985 @PaczkomatyPL @KrzysiekTSW1906 Świetna robota!: 1
@Stalowy_Szczur @PaczkomatyPL Nie zesraj się: 0
@mdxkrk @PaczkomatyPL Ale gdyby nie te negocjacjs to myślę że by nie doszło :): 0
@michal_trojan89 @PaczkomatyPL Dokładnie :): 1
Czy ktoś może poradzić jak wpłynąć na #inpost by łaskawie odblokował zepsuty paczkomat pełny świątecznych prezentów? Od 3 dni sezam nie do otwarcia,  call cent @PaczkomatyPL udaje że interweniuje, a paczki zapewne zobaczymy po #BozeNarodzenie: -2
Naprawcie bota @PaczkomatyPL https://t.co/rvn5gcajRO: 0
@PaczkomatyPL Mile panie zadzwonily i podaly kod odbioru 👍

Wesolych Swiat!: 1
@just_sokolik Pewnie się ktoś ucieszył z przedświątecznego prezentu za tyle kasy 🤡🤡🤡🤡 A @PaczkomatyPL co na to, że nagle zniknęło coś co było w ich paczkomacie: 0
@KrzysiekTSW1906 @norweskidrwal @PaczkomatyPL A brawurowa jazda Fiatem marki Cinqeucento ?? No i najważniejsze gwałty na zlecenie bez zlecenia?? 😅🤣: 0
@NhTs1985 @norweskidrwal @PaczkomatyPL Potrafią praktycznie wszystko Stefan 😎: 0
@norweskidrwal @PaczkomatyPL Widać że profeska 😎 @KrzysiekTSW1906 te twoje młode wilki tak potrafio ??
Zresztą moja paka też wczoraj 23 godziny 😁: -1
@norweskidrwal @PaczkomatyPL U mnie w jajo zrobili Mikołaja i teraz już tylko kurier.: -1
./out/36390fa9-5bba-43f4-99d2-e0c65cbd366e.zip is empty! No sentiment for value 81
@norweskidrwal @PaczkomatyPL Elegancko💪: no sentiment
I jest! 

Mniej niż 24 godziny .. ja pierniczę...@PaczkomatyPL jesteście zajebiści! https://t.co/xo7MLYfQNP: 0
@PaczkomatyPL A jeśli nie dotrze?: 0
Widzieliście kuriera @InPostPL1 który znalazł i oddał portfel dziewczynie, która zgubiła go pod @PaczkomatyPL ? Kolega miał dokładnie taką samą sytuację w @Uber Kierowca przywiózł mu portfel 20 km za Warszawę. Za darmo! Co ich łączy? Obaj to byli emigranci ciężko tu pracujący.✌🏻: 2
W ogóle to @PaczkomatyPL rozumieją mój nocny styl życia https://t.co/EISHsSsOxq: 0
@PaczkomatyPL Od wczoraj sprawa jest wyjaśniana. Paczka odebrana i głucho. Czas mija, paczka na święta nie dotrze. Więc mówicie, że dajecie gwarancję dostawy do świąt?: 1
RT @ZawszePomorze: @InPostPL1 brawo! @PaczkomatyPL Historia, którą żyje internet wydarzyła się w @gdansk 
https://t.co/D307zv1WoV: 0
16000 paczkomatów w Polsce,  3000 w Wielkiej Brytanii, 300 we Francji - to tegoroczna statystyka #InPost. Oprócz zasięgu może pochwalić się innowacyjnością i szybkimi dostawami. A to nie koniec ekspansji! #paczkomaty @PaczkomatyPL @RBrzoska @Komputer_Swiat https://t.co/lICrmDQHBq: 0
@PaczkomatyPL Ta informacja jest bledna. Infolinia nie jest w stanie wygenerowac i przeslac kodu.

Przetestowalismy to dla Was, i telefonicznie, i mailen, i formularzem na stronie.

No, chyba ze infolinia potrzebuje na to 30 dni🤥: 0
@PaczkomatyPL genialne :) https://t.co/NVFCRmAJO1: 1
@PaczkomatyPL Obiecanki, cacanki, a glupiemu radosc, jak mawiali nasi przodkowie.

Paczka to moze i na czas dotrze, lecz skrzetni pracownicy nie dali rady przyslac kodu odbioru. Paczka po 48 godz. wraca do nadawcy. Reklamacja trwa 30 dni🤣

Wesolych swiat!: 2
@uwteam Szkoda że na stronie @PaczkomatyPL nie można kupić takich opakowań. Chętnie sam bym nadawał paczki takie paczki.: 1
RT @ZawszePomorze: @InPostPL1 brawo! @PaczkomatyPL Historia, którą żyje internet wydarzyła się w @gdansk 
https://t.co/D307zv1WoV: 0
@PaczkomatyPL Zobaczymy czy to tylko kwestia aktualizacji statusów czy raczej tego, że z paczką nic się nie dzieje.: 1
@_TreworPerez @PaczkomatyPL @inpost No u mnie od 20 figuruje jako przygotowywana przez nadawcę, mimo że przez cały ten czas ma ją kurier.: 3
@InPostPL1 brawo! @PaczkomatyPL Historia, którą żyje internet wydarzyła się w @gdansk 
https://t.co/D307zv1WoV: 0
@PaczkomatyPL Super 3 dni zamiast 2 dni dostarczenia . Co mogę otrzymać w ramach zadośćuczynienie ??: 1
@InPostPL1 @PaczkomatyPL Czy wy się dobrze czujecie?? Czy wasi kurierzy sypiają?! Wszystko oky z wami?! 😳😳 https://t.co/8uTRcrqxFL: 0
@PaczkomatyPL prawda to, że statusy się aktualizacja z dużym opóźnieniem ze względu na ruch świąteczny? Jeden sklep mi powiedział że towar wysłał a u Was w aplikacji mam ciągle informacje, że paczka w przygotowaniu przed nadawcę 🤔: 2
@PaczkomatyPL ... a na dostarczenie paczki reklamujecie  2 dni ? To ku... jakiś żart. Nie mamy pańskiego płaszcza.... . Znacie to ?: 0
@PaczkomatyPL Reklamacja poszła o  11 dnia dzisiejszego: 0
@PaczkomatyPL Po -miu godzinach od raklmacji z powodu zastoju  po 47 godzinach oczekiwania na odebranie z paczkomatu?: -1
./out/14ff106b-0480-47e8-a375-89f644f6d161.zip is empty! No sentiment for value 103
@Cbasiaaa @PaczkomatyPL #replay @inpost https://t.co/iFiNJFk2Jk: no sentiment
@PaczkomatyPL Ciężko wam to idzie 🧐🤔🤥 https://t.co/EqgQ5mEGlx: 0
@PaczkomatyPL Co w związku z tym? Zamieszczacie wpis o gwarancji dostawy przed Świętami, a tymczasem po prostu nie zmieniacie statusu przesyłki pomimo jej odebrania przez 2 dni.: 1
@PaczkomatyPL Posłałem na DM: 0
@PaczkomatyPL co z Wami? Odebraliście paczkę w poniedziałek i nadal jej nie dostarczyliście :( Utknęła w oddziale.: 0
@PaczkomatyPL &lt;3 yo myslalam ze nie dostarczycie paczki przed swietami: 0
./out/272b93f0-0398-4889-bb17-978728ada440.zip is empty! No sentiment for value 109
@PaczkomatyPL Aha oki: no sentiment
@PaczkomatyPL Zakończyła się dostarczeniem paczki. Dziękuję: 0
@PaczkomatyPL Martwi mnie ten przecinek na samym początku…: 0
@PaczkomatyPL Nie można się doczekać na połączenie.. 10 minut czekania i nadal głucho 🤔: -3
@PaczkomatyPL 📣 właśnie zmienił się status 😉 Po 4 dniach oczekiwanie paczki zostały przyjęte w oddziale, mam nadzieje że jak najszybciej ruszą w drogę, bo @RylMateusz przebiera nogami 😀
Życzę Wam aby jak najszybciej się Wam odkorkowało w magazynach 😉: -1
@PaczkomatyPL Już doszła: 0
@PaczkomatyPL Srututu tu, dzwoniłem 10 razy i za każdym razem niby złe cztery ostatnie cyfry podawałem 😐: -2
./out/cff99efe-aa39-4943-933d-81b524bcdf01.zip is empty! No sentiment for value 116
@PaczkomatyPL: no sentiment
@PaczkomatyPL A to ciekawe. Kurier InPost odebrał paczkę ze sklepu 2 dni temu, a ona do tej pory figuruję w systemie jako nieodebrana od nadawcy🤔: 1
@PaczkomatyPL Czy można defekować do paczkomatu?: 0
Dziękuję , i ledwo to wyjołem @PaczkomatyPL https://t.co/s2vynDdcXV: -2
@niewiarygodnie @PaczkomatyPL to najważniejsze!: 1
./out/adfbc786-bc52-4490-bc8a-98736ca3aba9.zip is empty! No sentiment for value 121
@Poprostu9 https://t.co/UBUmrV2WlJ: no sentiment
@fhusthacja @PaczkomatyPL Maybe, ale po tym jak to napisałam to wszystko zaczęło nagle działać haha: 0
@PaczkomatyPL Ale oki: 0
@PaczkomatyPL Dostarczycie i będzie ok. Do piątku mam nadzieje, że się uda.: 1
@PaczkomatyPL Właśnie idę po jedną a na drugą czekam xD: 0
I to jest świetny news na koniec roku! @PaczkomatyPL wprowadza opakowania wielokrotnego użytku do swoich paczkomatów. Brawo #Inpost! 👏👏👏

#gocircular #ecommerce #greenlogistics

https://t.co/Q4d6aTb2wc: 0
@PaczkomatyPL piękny musicie mieć zapiernicz… Infolinia nie funkcjonuje, a moje cztery paczki wyciągnięte przez kuriera z paczkomatu w sobotę niewiadomo gdzie są…: 1
@Rojek333 @PaczkomatyPL Imo juz dziś nie będzie unlucky a jutro nie wiem czy ja odbiorę bo nie wiem czy będę na chacie: -2
@MATL01285388 @PaczkomatyPL nie wiem ale mysle ze ciezko: 0
./out/8c2734cb-d61e-42d7-8d44-444d6ca07b51.zip is empty! No sentiment for value 130
@Rojek333 @PaczkomatyPL dzis juz nie bedzie: no sentiment
@MATL01285388 @PaczkomatyPL no bo kazdy bierze z samego rana jak nikt jej nie wzial to nie wiem czy dzisian bedzie: -1
@Rojek333 @PaczkomatyPL kurwa 5 godzin czeka XDDD: -1
@MATL01285388 @PaczkomatyPL DEBIL XDDD ona lezy na magazynie teraz jak ktos przyjedzie po paczki na magazyn to ja wezmie: -1
@PaczkomatyPL sprawa już rozwiązana, to była wina sklepu 🖤: 0
juz 5h dzieki @PaczkomatyPL za wspaniała obsługę https://t.co/ST15Uas9wo: 1
@PaczkomatyPL Jaką karę sobie wymierzycie za niedopilnowanie obowiązków przewoźnika?: -1
@niewiarygodnie @PaczkomatyPL Pewnie świąteczne przeciążenie.: 0
@fhusthacja Mi nie chce wysłać kodu weryfikującego a muszę przedłużyć pobyt paczki w paczkomacie 
@PaczkomatyPL  co jest nie tak z aplikacją?: 0
@PaczkomatyPL moja paczka jest wydana do doręczenia od wczoraj rana a nadal do mnie nie trafiła 🙄 kiedy mogę oczekiwać doręczenia?: 0
@PaczkomatyPL Proszę, nie pozwalajcie waszym kurierom wciskać ponad gabarytowych przesyłek na siłę do paczkomatu bo później się ich nie da wyciągnąć. W środku paczki był słoik który mógł pęknąć. Nie powinniście wcale przyjmować takich paczek. https://t.co/vcJyn9dEXd: -1
@PurpleG14850464 @PaczkomatyPL Też mam ciągle pudło, to chyba tak celowo jest :(: -1
@PaczkomatyPL I o ile 22.12.2021 są też dostarczane??: 0
@PaczkomatyPL Czyli 23.12.2021 też dotrą ?! Czy nie dostarczane w tedy są?!: 0
./out/ab53f184-93e1-4177-8c14-4aa153576e95.zip is empty! No sentiment for value 144
@PaczkomatyPL 605806746144230023486625: no sentiment
./out/670b2fc2-1497-415d-8a00-eef565a7a8ed.zip is empty! No sentiment for value 145
@PaczkomatyPL dziękujee: no sentiment
@PaczkomatyPL @_Sorin_Bitch_ Kurierzy paczki potrafią zgubić
Taka sytuacja wydarzyła się przed moim domem: 0
@PaczkomatyPL @Inpost @paczkomaty
Na wysokości mojego domu kurier zgubił kilka paczek
Dodzwonić się do kogokolwiek jest niemożliwe: -1
Jak fajnie przeczytać jakieś pozytywne wiadomości! Lajk dla @PaczkomatyPL !

https://t.co/TwXCqyb2Db: 2
@PaczkomatyPL 
Może ogarniecie swoje paczkomaty, żeby nie odpierdalały takich numerów, jak mi wczoraj?
Takiej chały to nawet poczta nie odpierdala.: -1
@PaczkomatyPL Ja właśnie niedługo dostanę 2 paczki od was! Tylko jest problem, wzięłam już 25 razy udział w loterii InPost i za każdym razem było"Ehhh… pudło!": -2
@Mishotpl @onestepcloserxv @PaczkomatyPL Ja zawsze musiałam oba podawać abo moze już po prostu aplikacji wcześniej wpisałam, tak czy siak adres zamieszkania i tak musiała dodawać i to zawsze i to było podpisywane jako „adres rozliczenia” czy coś takiego: 0
@PaczkomatyPL Widzę że macie nowy standard doręczeń - kliencie czekaj pod paczkomatem :-) https://t.co/sPElHO76FN: -1
Hej @PaczkomatyPL @RBrzoska  czy to jakiś nowy standard doręczenia #inpost w Suwałkach ? https://t.co/jFDvNj4Nca: 0
@ranpokrul @cpun69 @PaczkomatyPL @PolskaPolicja I KURWA INPOSTU JAPJERDKEEEE NIEE: -1
@ranpokrul @cpun69 @PaczkomatyPL @PolskaPolicja Zegnam: 0
@ranpokrul @cpun69 @PaczkomatyPL @PolskaPolicja jedyne psy jakich ise boje to psy pasterskje moicj sonsiaduw owczarki podhalanskje harnas i dolina. Az ciarki czlowjeka przechodzom: 0
@ranpokrul @cpun69 @PaczkomatyPL @PolskaPolicja KAPUS KAPIS INACZEJ SZESDZIESIONYKA: -2
@ranpokrul @cpun69 @PaczkomatyPL hlopie  jakim winem ja swojom setke pzyniose. 😉: 0
@ranpokrul @cpun69 @PaczkomatyPL DOKLADNIE CHCE POW ZE BDMY SIE NAPIERDALAC POJUTRZD PO PASTERCE: 0
@PaczkomatyPL prosze mi to wyjaśnić https://t.co/D7YkKQHYII: 0
@PaczkomatyPL @wladcazuli Jesteście bezkonkurencyjni🙂🙂: 1
@fjentoa @onestepcloserxv @PaczkomatyPL Tylko e-mail, nr tel nie musi być (tak, wiem, jest wymagany), a adres paczkomatu: 0
@PaczkomatyPL jak na was k szanuje https://t.co/xcw7fUWqNA: 0
@Morunskyy @PaczkomatyPL No zeszły niestety, a gadżeciarnia nie dała rady przed świętami dosłać nowych...: 1
@Dejw09 @ZKierowca @PaczkomatyPL @Allegro_Group Dalej korzystam z żółtego🤔: 0
@PaczkomatyPL Tyle to wyczytałem sam z śledzenia :/ Niczego nowego mi nie powiedzieliście.
Pytanie dlaczego to tak długo trwa? Szybciej przywiózł bym sobie je sam...: 1
@PaczkomatyPL Dwie moje przesyłki utknęły
653999410031801004356238
631358646129100114802740
Co się z nimi dzieje? Co wy robicie w tych oddziałach?! Odległość między miastami 100 km a przerzucacie sobie paczki między oddziałami i nie możecie przez dwa dni dostarczyć. Kpina!: 0
@Mishotpl @onestepcloserxv @PaczkomatyPL Trzeba podać numer telefonu i adres mimo wszystko: 0
@fjentoa @onestepcloserxv A np przez @PaczkomatyPL: 0
@PaczkomatyPL Nie macie się czym szczycić !!! Gdzie moja paczka dziś miała dojść a utknęła między oddziałami !!!: -1
@PaczkomatyPL Czyli wg was czarny widać tak samo wyraźnie, jak biały? Bo ja nie o neutralności piszę, tylko o oczojebności. No ale dzięki za odpowiedź, obrona musiała być, szkoda, że chujowa.: 0
@PaczkomatyPL Chyba pora zmienić kuriera w moim rejonie. Drugiego tak chamskiego, wrednego i narwanego kuriera jeszcze nie spotkałam. 
Zresztą to nie pierwszy raz kiedy spotkałam się z takim jego zachowaniem: -3
@PabloSabre @PaczkomatyPL Bez problemu, szkoda tylko że bez otwieracza 🙄: 0
RT @PaczkomatyPL: Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszyst…: 1
@PabloSabre @PaczkomatyPL Spoko mam nadzieję że delord nie nasral mi do paczki: 2
@Morunskyy @PaczkomatyPL Koks. I jak, wszystko git? 🦍: 0
@gustawo_69 @Aleksan46513793 @PaczkomatyPL Każdy pretekst jest dobry: -1
@CarlossDee @PaczkomatyPL W piątek o 17 wkładałem paczkę do paczkomatu, w sobotę ok 10 była już w Świdniku pod domem: 0
RT @szeptunka: Dostałam informację od @PaczkomatyPL, że mój paczkomat jest "przepełniony" i w związku z tym moja przesyłkę pozostawiono w i…: 1
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
./out/650059a8-1ea3-4bd4-8805-5852e4cda81f.zip is empty! No sentiment for value 181
@CarlossDee @PaczkomatyPL https://t.co/xovLWai5f9: no sentiment
Jeszcze 6 ostatnich paczek zamówionych w weekend i wczoraj. I dziś wszystko w paczkomacie. Naprawdę rewelacja @PaczkomatyPL 👍👏 https://t.co/CNntZDliDT: 1
./out/1dca4a72-271b-4e8b-864e-3402aff36cf3.zip is empty! No sentiment for value 183
@PaczkomatyPL: no sentiment
Się LOVELKOWCY zdenerwują, że im blachosmrody z @PaczkomatyPL @InPostPL1 Kurier staje na ścieżce i chodniku 😶😶🤣🚧 https://t.co/FPdFKR2q3d: 0
@PabloSabre @PaczkomatyPL Ja już mam dzięki wielkie 🤠: 0
@ZKierowca @PaczkomatyPL @Allegro_Group Wcześniej były żółte. Jest jakiś progres.: 1
Kurier @PaczkomatyPL dziś miał wydane 3 przesyłki do doręczenia do paczkomatu. Doręczył dwie. Jaka nadal wisi w doręczeniu? Fabric case do Pixela 5. Klasyg.: 0
@Gruszka @PaczkomatyPL Słuszna decyzja Marcin, bo w @PaczkomatyPL wcale nie jest tak różowo, jak deklarują. https://t.co/OeiGNcYqSb: 0
RT @PabloSabre: Wszystkie zamówienia oficjalnie opuściły nasz magazyn. Teraz Prayge żeby to wszystko wam doszło na Święta, bo my się staral…: 1
Wszystkie zamówienia oficjalnie opuściły nasz magazyn. Teraz Prayge żeby to wszystko wam doszło na Święta, bo my się staraliśmy jak najszybciej to ogarnąć. @PaczkomatyPL - dajcie z siebie wszystko XD https://t.co/BbZoYAiqwx: 1
16 000 paczkomatów &lt;3 Brawo @PaczkomatyPL !

https://t.co/Ux9FDc3yvd: 0
@debianeq @MaciejSulima @PaczkomatyPL w sensie im dalej od sortowni tym większa szansa że kurier oddaje paczki rano następnego dnia, kiedy obiera nowe paczki (żeby nie jechać do sortowni dwa razy w ciągu dnia): 0
@debianeq @MaciejSulima @PaczkomatyPL zależy skąd wysyłasz, bywa że i w normalne dni (nie przed świętami) kurierzy jeżdżą z paczkami aż do następnego dnia... z sortowni powinno pójść już normalnie, co najwyżej nie trafią do stacjonarnego paczkomatu tylko będą do odbioru w mobilnym paczkomacie...: 1
@Piechocinski @PaczkomatyPL to błogosławieństwo. Dzisiaj miałem otrzymać przesyłkę z lekami przez @dpd_polska . Pomimo, że z żoną jesteśmy na izolacji kurier zaznaczył, że nie było nas w domu.: 0
Te @PaczkomatyPL nie byłyby znów takie złe, gdyby tylko nie były takie oczojebne białe. Brawo dla @Allegro_Group, że aż tak bardzo nie spieprzyli krajobrazu.: -1
Już tu jest! Paczkomat #16000 wyposażony w czujnik powietrza stanął w Gorzowie Wielkopolskim - mieście partnerskim programu Green City. Mamy nadzieję, że do nowej maszyny trafi mnóstwo prezentów dla Was i Waszych bliskich 🎁 https://t.co/VlYNN9wkPw: 1
@UPSPolska @PaczkomatyPL W sumie to macie przecież punkty odbioru. Jak nie radzicie sobie z obsługą przedświątecznego ruchu to może zrzucać do punktów odbioru. Macie przecież takie?: 0
@benchmarkpl @PaczkomatyPL @Allegro_Group OSA nie wyrabiał się z wysyłką na czas już w zeszłym tygodniu mając długie czasy dostawy i chyba po prostu @Allegro_Group postanowiło nie robić z siebie durniów i zrobić tak aby wszyscy sprzedawcy również mieli dramatycznie długi czas dostawy.: -2
@benchmarkpl @PaczkomatyPL @Allegro_Group Zamówienia składane około 16 były wczoraj wysyłane pomimo gwarancji wysyłki tego samego dnia dla zamówień składanych do 13:30. Dzisiaj te zamówienia już przed południem zaczęły pojawiać się do odebrania w paczkomatach.: 0
@benchmarkpl W naszej firmie wysyłamy bez problemu na bieżąco. Jesteśmy przygotowani. @PaczkomatyPL również zapewniając dostawę do świąt przesyłek nadanych nawet 22 grudnia natomiast @Allegro_Group złamało zasady obliczania czasu dostawy traktując 22 i 23 grudnia jako święto.: 4
@UPSPolska Ile razy w miesiącu robicie wysyłki?
Bo z tego co widzę to się nie załapałem na ten właściwy dzień. Mam paczkę, która ponad tydzień wędruje przez magazyn w Mirkowie. W sumie to bym podjechał i pomógł. W sumie nie polecam. Nie jest to @PaczkomatyPL 🤦‍♂️
Dramat trwa. https://t.co/JekxOikUc0: 0
#InPost uruchomił Paczkomat® nr 16 000. Jubileuszowe urządzenia znajduje się w Gorzowie Wielkopolskim. W sumie sieć Paczkomatów liczy już 19 300 urządzeń w trzech krajach. @PaczkomatyPL 

https://t.co/eCjJvKOt16: 0
#InPost zmienia praktyki po interwencji @UOKiKgovPL. Co się zmieni?
@se_pl @PaczkomatyPL https://t.co/Ss2rRboWBJ: 0
@BIPolska Siła #ecommerce jest https://t.co/GcIPBbnD4x. w dostawie szybkiej, łatwej i przyjemnej + optymalnej cenowo. Odnoszę wrażenie że https://t.co/GcIPBbnD4x. za sprawą @PaczkomatyPL mamy tutaj mocne popularyzowanie się.: 5
Nie... po czwartą paczkę nie wróciłem. @PaczkomatyPL  Jeśli chcecie rozładować paczkomaty to moze nie wysyłajcie 3 powiadomien o dostarczeniu jednej paczki. Wystarczy jedno pozwalajace odebrać wszystkie paczki.: 1
@InPostPL1 wprowadza trzy istotne zmiany, które dotyczą: przekierowania paczek do innych paczkomatów, wydłużenia czasu odbioru oraz dopłat za weryfikacje rozmiaru przesyłki 👌🏼 

@PaczkomatyPL 
https://t.co/xyJzxAVINn: 1
@PaczkomatyPL  4 przesyłki zostały dostarczone do paczkomatu. Otrzymałem powiad. o pierwszej. odebrałem. wrociłem do samochodu. otrzymałem powiadom. o drugiej. odebrałem. wyjechałem z parkingu. otrzymałem 3 powiadom.. odebrałem.  proszę zgadnąć co się stało 20 min. później?: 0
@Gruszka @PaczkomatyPL Jakim cudem większość przesyłek wychodzi z dnia na dzień, kurierzy gwarantują dostarczenie przed świętami paczek wysłanych 22 grudnia, a allegro z dupy sobie postanawia ustanowić 22 i 23 grudnia świętem?: 4
@AmazonHelp Dzień dobry, już wiem, że z winy przewoźnika @PaczkomatyPL paczki nie otrzymam. Bardziej z winy kuriera który się obraził i dwa razy zaznaczył, że mnie nie ma w domu a fizycznie czekaliśmy na niego. Miał pecha bo go widzieliśmy. Poszła reklamacja. Paczka wraca do nadawcy. https://t.co/mqc3rhM6GK: -3
@debianeq @PaczkomatyPL Dojdą.: 0
@ChLiberator @PaczkomatyPL No martwię się, no wysyłałam tym osobom paczki. I chciałabym żeby doszły przed świętami 🙄: 1
@debianeq @PaczkomatyPL Kolejna… 🤦🏻‍♂️😁: 0
@PaczkomatyPL Status się już zmienił :): 0
@debianeq @PaczkomatyPL Prezenty? Po świętach!: 1
@debianeq @PaczkomatyPL Z auta odbiór może być. Jak co święta.: 0
@debianeq @PaczkomatyPL Święta🙃🥴: 0
@plipiec @Gruszka @PaczkomatyPL Same Day dalivery ze sklepu Allegro z piątku jeszcze nie dotarło do mnie.: 0
@PaczkomatyPL Było ich dużo, dzięki ;)): 0
RT @benchmarkpl: @PaczkomatyPL zmieniają się na lepsze i jest ich coraz więcej. Dobra robota, InPost

https://t.co/HtlK9rJkY8: 0
@JakubWoch2 @PaczkomatyPL No wiem, ze kurierzy mają teraz zapier …

Ale słabo trochę :(: -1
@MaciejSulima @PaczkomatyPL Lipa :/: 0
@debianeq @PaczkomatyPL Święta idą :p: 0
@debianeq @PaczkomatyPL Nie dostarczył do oddziału, trzyma je na samochodzie. Do świąt nie dojdą do odbiorcy na 95%.: 1
./out/be49496d-3a45-4824-8029-13c1c84b5348.zip is empty! No sentiment for value 224
@PaczkomatyPL https://t.co/AnYZHzdEiO: no sentiment
RT @PaczkomatyPL: Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszyst…: 1
@Kejtx1 @PaczkomatyPL Ja obstawiam z 60 lekko: 2
@corgimichal @PaczkomatyPL Statystyki mogą przerazić 🙈🙈🙈: 0
Targ działa, ciągle realizujemy mnóstwo Waszych zamówień. Jedyna zmiana to fakt, że nie możemy zagwarantować dostawy, ale ciągle jest na nią szansa jak zapewnia na przykład nasz partner @PaczkomatyPL https://t.co/kHURCjWgaj: 2
RT @corgimichal: @PaczkomatyPL może zróbcie takie podsumowanie roku jak spotify np ile paczek się odebrało ile się nadało ile czasu spędził…: -1
./out/b46d4f17-76a1-4ffe-874e-62e4320c8f71.zip is empty! No sentiment for value 230
@corgimichal @PaczkomatyPL takkk!: no sentiment
@PaczkomatyPL może zróbcie takie podsumowanie roku jak spotify np ile paczek się odebrało ile się nadało ile czasu spędziło się przy paczkomacie ile CO2 się zaoszczędziło wybierając paczkomat itp 

Chciałbym zobaczyc ile paczek odebrałem w tym roku: -1
E-handel a przyszłość mobilności miejskiej

#Motoryzacja #PełnySerwis #Aktualności #Elektromobilność @_PSPA @PaczkomatyPL @amazon 
https://t.co/HoKe7Bm5h2: 0
@GadowskiWitold @Malvineaa @PaczkomatyPL są w przystępnej cenie i wszędzie dostępne no chyba że zaczną stosować w firmie #segregacjasanitarna: 2
@PaczkomatyPL zmieniają się na lepsze i jest ich coraz więcej. Dobra robota, InPost

https://t.co/HtlK9rJkY8: 0
@Allegro_Group ŁŻECIE. Nawet dziś @PaczkomatyPL kolejny raz dały gwarancję dostawy przed wigilią paczek nadanych do 22 grudnia włącznie. Wczorajsze zamówienie złożone około 16 było przed godziną 12 doręczone do paczkomatu. Nadajecie się do pozwu zbiorowego za działanie na szkodę. https://t.co/LDZVsN36D9: -3
@PaczkomatyPL upadliście na głowę? Warszawa, Gagarina 6 https://t.co/UMh8zo7IDa: -1
./out/73362228-136f-4f27-9c41-d839cca69109.zip is empty! No sentiment for value 237
@PaczkomatyPL 630552656121500014263024: no sentiment
@ald_lip @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Urbanistyka paczkomatu to nie tylko kwestia przejścia ale też kwestie dostaw, a także tego kiedy i ile ludzi odbiera paczki i czego potrzebują - chodzi o chodnik, drogę, doświetlenie, kwestie bezpieczeństwa (przechodzenie przez jezdnie, rozbój/napad), Hałas związany z używaniem: -1
@ald_lip @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Właśnie chodzi o to że blokują przestrzeń, ale nie w tym zresztą rzecz - przestrzeń nie ekst do nich dostosowana. Skoro nieprędko znikną, to warto byłoby zastanowić się jak je dostosować do przestrzeni i jak dostosować przestrzeń do nich: 0
@PaczkomatyPL Zgubiliście mi paczkę 👍: 0
RT @wdrodzekudobru: A w związku jeszcze z tą paczką, to zamówiłem ją do @PaczkomatyPL i pierwszy raz odebrałem ją zdalnie.

Powiem Wam, że…: 0
RT @wdrodzekudobru: Druga paczka z @PaczkomatyPL odebrana i to drugi raz zdalnie za pomocą aplikacji. Polecam 🙂

#polecajki https://t.co/ui…: 0
RT @wnppl: .@PaczkomatyPL uruchomił w Polsce paczkomat nr 16 000. Stanął on w Gorzowie Wielkopolskim. Ponadto firma postawiła paczkomat nr…: 2
.@PaczkomatyPL uruchomił w Polsce paczkomat nr 16 000. Stanął on w Gorzowie Wielkopolskim. Ponadto firma postawiła paczkomat nr 3000 w Wielkiej Brytanii, a we Francji, gdzie InPost działa pod marką Mondial Relay, spółka postawiła paczkomat nr 300 https://t.co/oGx3pcUu1L: 2
RT @PaczkomatyPL: Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszyst…: 1
RT @PaczkomatyPL: Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszyst…: 1
@PaczkomatyPL please, znajdźcie moją paczkę: 1
Święta Bożego Narodzenia nadchodzą wielkimi krokami! Jeśli świąteczne zakupy wciąż przed Wami, to przypominamy, że wszystkie paczki nadane do środy 22 grudnia (włącznie) zostaną objęte gwarancją dostawy InPost, co oznacza, że trafią do swoich adresatów maksymalnie w Wigilię!🎁 https://t.co/Y3HV4V7AnQ: 3
@WojtekKardys @PaczkomatyPL @InPostPL1 Mordo, widząc źle zaparkowanego golfa wołasz Volksvagena? ;)
To zarządzca terenu postawił tam paczkomat: -1
Czy ktoś w @Allegro_Group ma zamiar zareagować na skandalicznie i sztucznie wydłużony czas dostawy, który jest widoczny od wczoraj? Sprzedawcy wysyłają zamówienia z dnia na dzień @PaczkomatyPL gwarantują dostarczenie przed świętami natomiast oferty mają czas dostawy 7dni!!??: 7
@PaczkomatyPL Już waszą infolinia mi powiedziała, że mogę się w dupę pocałować, Pierw mówili że paczka jest procesowana prawidłowo i będzie do odbioru max 20.12 Nagle okazuje się że paczka jest źle procesowana i mogę się kontaktować z nadawcą. Kpina nic więcej: 2
RT @genesispr_: #InPost wita w firmie Dagmarę Brzezińską, nowego członka zarządu. Będzie odpowiadać za sprzedaż krajową, klientów międzynar…: -1
#InPost wita w firmie Dagmarę Brzezińską, nowego członka zarządu. Będzie odpowiadać za sprzedaż krajową, klientów międzynarodowych, SME, obsługę klientów, procesy doskonalenia sprzedaży, wdrażanie nowych produktów. @PaczkomatyPL @RBrzoska @dlahandlupl https://t.co/MZIq5vR05w: -2
Czy Mikołaj w tym roku wejdzie przez komin czy ...? Firma @InPostPL1 jest liderem w Polsce w kat. @PaczkomatyPL, choć ta kategoria jest zastrzeżona znakiem towarowym "Paczkomat" tylko dla niej. A kształt paczkomatu? Również, tyle że wzorem przemysłowym ➡️ https://t.co/gwH3qYvz4Z https://t.co/QVkEiDjqpE: 1
Kurierzy mają jeszcze kilkanaście milionów paczek do dostarczenia przed Wigilią. Dzielnie wyręczają w tej pracy świętego Mikołaja | @PaczkomatyPL @PocztaPolska @Allegro_Group @PKN_ORLEN https://t.co/1i1SEKZpzd: 1
@PaczkomatyPL @xkom_pl Dziwne, bo z moją polecieliście w chuja https://t.co/ic058Vi8LS: -1
Generalnie to jebać InPost.
&gt; 3 dni czekasz na paczkę
&gt; Wypisujesz na infolinię co się dzieje z paczką
&gt; Niestety nie możemy panu pomóc, paczka powinna być dziś, max jutro 
&gt;Chuj ci w dupę, zwracamy paczkę do nadawcy 
#inpost #paczkomatypl #paczkomaty https://t.co/iGY7MOY5g5: -2
@PaczkomatyPL Zajebiści jesteście, oby tak dalej!  3 dni czekania na paczkę, Pytania na infolinię, co się dzieje z paczką - otrzyma ją pan dziś max jutro, kilka godzin później zwrot do nadawcy. To jest po prostu jakiś żart https://t.co/kPhHZ0FNEP: -2
RT @wdrodzekudobru: 4/11/2021

Wyszedłem ze śmieciami i akurat przyszedł mi SMS, że paczka z książkami z wydawnictwa @oazapl w @PaczkomatyP…: 0
RT @wdrodzekudobru: Przyszły i w piątek odebrałem!

Po raz kolejny dziękuję @PaczkomatyPL za skorzystanie z Waszych usług 🙂🙏🏼
#paczkomatyin…: -1
@SialaTV @PatrykSlowik @PaczkomatyPL I to jest najlepsze rozwiązanie.: 1
@PaczkomatyPL uswiadomcie swoich pracownikow ze niscy ludzie tez istnieja u jak daja wysoko paczke to niech nie wpychaja jej tak gleboko tylko z brzegu wystarczy, bo nie siegam a po paczkomacie sie nie wespne i jest wtedy problem: 0
RT @Darek_tech: Ważne i dobre zmiany w #PaczkomatyPL👌 Szczegóły od @gsmManiaK... https://t.co/a9Fgu9sOIP👀
#InPost #przesyłki #kurier #zakup…: 2
Ważne i dobre zmiany w #PaczkomatyPL👌 Szczegóły od @gsmManiaK... https://t.co/a9Fgu9sOIP👀
#InPost #przesyłki #kurier #zakupy https://t.co/HTDR6ywwHE: 2
@PaczkomatyPL Jestem zła!!! Gubicie a potem przetrzymujecie moją paczkę na magazynie, jutro kończy się ważność etykiety Allegro i nikt, absolutnie nikt nie chce się sprawą zająć. Piękna obsługa klienta 🤬🤬🤬: -4
@PatrykSlowik Ja tylko przez @PaczkomatyPL: 0
./out/e7ae0073-5a1a-4bf2-a6aa-e16744fdf3d2.zip is empty! No sentiment for value 267
@PaczkomatyPL https://t.co/NIJJiiQXqE: no sentiment
@PRKP99 @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ A możesz przybliżyć? Jeśli nie blokują przejścia i są w miarę ładne, jakoś dostosowane to co jest w nich złego?: -1
Dziękujemy, że jesteście z nami i doceniacie pracę jaką wykonujemy. Życzymy, abyście zawsze byli usatysfakcjonowani z usług InPost i obiecujemy, że zrobimy wszystko, aby jeszcze lepiej wykonywać swoje zadania. Bądźcie #OutOfTheBox

Wesołych Świąt! 🎄 https://t.co/NppXFRYG2y: -5
@PaczkomatyPL 602675946101780015674648 to numer mojej paczki która nie dotarła ( wbrew zapowiedziom) do punktu odbioru. Proszę o informację co się stało ?: 0
@PaczkomatyPL Dobrze, dziękuję ❤️: 0
@PaczkomatyPL Na podpiętych zdjęciach wyżej  jest wszystko. Daty, godzina, numer przesyłki. Proszę jeszcze raz podpinam https://t.co/yV2GBcPcjL: -1
@PaczkomatyPL Oki zrobiłam, dałam dostęp do wszystkiego mam nadzieję ze zadziała, być może telefon mi coś poblokowal, ale ogólnie powiadomienia były włączone https://t.co/7YGnfVd88S: 3
@PaczkomatyPL napisałem DM i będę wdzięczny za odpowiedź ;-): 2
@PaczkomatyPL @AmazonNewsPL dojdzie. A nie chce za każdym razem czekać, zgłaszać reklamacje i walczyć z kurierem.: -1
@PaczkomatyPL Ja: złoże reklamację (zgłosiłem telefonicznie). K: bujaj się kurwa i się Pan kurier rozłączył. Bardzo milutko. Ogarnijcie Pana bo Pan się obraził, za poprzednie paczki i wynikające z tego sytuacje (DM). Proszę mi dostarczyć z @AmazonNewsPL paczkę do 23.12 bo teraz nie wiem czy: -2
@PaczkomatyPL Hej @PaczkomatyPL rozumiem, że już moje paczki zawsze będą ze statusem "kolejna próba doręczenia" ?Znowu nie dostarczył mimo, że jestem w domu tak jak w sobotę. Dzwonię do kuriera pytam grzecznie dlaczego? K: nie było cię w domu. Ja: Byłem. K: No i co mi zrobisz. https://t.co/tCDokWFT8Z: 0
#Olsztyn @PaczkomatyPL @InPostPL1 #Inpost #swieta2021

https://t.co/Xne1mnJb7z: 0
@Mordora_ @PaczkomatyPL Paczka po tym czasie jest zwracana do nadawcy, więc wydaje mi się, że kontakt ze sprzedawcą tutaj pomoże bardziej: 0
Zapomniałam odebrać paczki z paczkomatu, a był tam prezent dla brata xD Tak, wiem, że miałam 2 dni do odbioru. Tak wiem, że mogłam przedłużyć. Jestem największą pepegą w moim otoczeniu. 
@PaczkomatyPL pls help: 0
Kolejny numer próbujący oszustwa poprzez fikcyjne zakupy @PaczkomatyPL #olx https://t.co/csev9pjQ0o: 1
@PaczkomatyPL  dlaczego nie przychodzą mi powiadomienia w aplikacji? Paczka jest do odbioru w paczkomacie a ja żadnej informacji nie dostałam, ani SMS, ani mail, ani powiadomienia w apce, dopiero teraz dowiedziałam się że paczka jest do odbioru juz od 2 godzin: 0
@Pseudonim_Nel @PaczkomatyPL Oczywiście, że się uda 🤙 innej opcji w ogóle nie biorę pod uwagę 😈🌞⛳: 0
@tomek_lebel_68 Hej @PaczkomatyPL uwielbiam Was i trzymam kciuki ✊ żeby wszystkie prezenty dotarły, nie tylko te moje 😊

Tomek, jak sądzisz? Uda się? 😁😜: 2
@PaczkomatyPL dostarczyły mi kolejną przesyłkę zagraniczną &lt;3 i pewnie następną tuż przed świętami dostanę :)
@PocztaPolska nie chce dostarczyć paczki z sierpnia, widocznie komuś bardzo WIBRATOR się spodobał że sobie przywłaszczył. Na zdrowie, bo edu seksualnej w Polsce brak..: 1
InPost cieszy się niesłabnącą popularnością wśród klientów, jednak nie brakuje też narzekań na niektóre praktyki firmy. Zajął się tym @UOKiKgovPL, zmuszając @PaczkomatyPL do wprowadzenia zmian
https://t.co/CCxS3Kgr8O: -3
📦 #InPost @PaczkomatyPL uruchomił właśnie pilotażowy program wtórnego obiegu opakowań używanych w handlu elektronicznym.✅

⬇️Opakowanie zostanie zwrócone do magazynu, skąd trafi do e-sklepu, a firma wykorzysta je ponownie do wysyłki.⬇️@GrupaCCC

🔗 https://t.co/omaxapiKoO: 0
dear @PaczkomatyPL, this is an example of bad UI: you should remove the arrows from the box size selector and possibly enlarge the "white dot". The way it is done now, we see 5 CTAs, but only 2 work: https://t.co/DKfgMeMyJj: -8
@PaczkomatyPL - co się zmieniło dla konsumentów w wyniku postępowania prezesa #UOKiK?

https://t.co/ByOGh8K61L: 0
@PaczkomatyPL @InPostPL1  paczkomaty na Łazarzu i Wildzie be like.... https://t.co/3Wlyj39pqK: 2
@ald_lip @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Obłożenie chwastami nie zmienia zbyt wiele jeśli chodzi o organizacje przestrzeni, a o to chodzi w podnoszeniu problemów urbanistycznych jakie paczkomaty powodują.: -1
zazwyczaj tutaj narzekam na @PaczkomatyPL. Dzisiaj jestem mega pozytywnie zaskoczony 😱 co za zmiana 👏🏼 zobaczymy czy kolejne przesyłki też tak szybko dojdą 🤔: -1
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Czy wolno srac do paczkomatu?: 0
Ciekawa historia sukcesu. Wolę nasze lokalne zwycięstwa jak np. @PaczkomatyPL ale że zajmuje się #marketingautomation to poznanie @HubSpot od strony c-level również jest warto uwagi. https://t.co/sRnHKgeoqX: 2
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Ej, ale akurat w tym miejscu paczkomat nikomu nie przeszkadza, a jest bardzo przydatny, zwłaszcza w czasie pandemii. A to obiekt tymczasowy, łatwo potem zlikwidować lub przenieść.: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Uchwała krajobrazowa dotyczy reklam, ogrodzeń i małej architektury. Paczkomaty to tymczasowe obiekty budowlane, więc uch.kraj. może wpłynąć tylko na reklamy na nich, a nie na ich sytuowanie.: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ i bardzo dobrze, niech ich będzie jak najwięcej: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Ej temu miejscu zaszkodziłby tylko napalm. Tu jest już wszystko.: -1
@PRKP99 @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ https://t.co/celCGvkwDZ: 0
@ald_lip @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Co to znaczy?: 1
@ald_lip @PRKP99 @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Zielone są Allegro.: 0
@PRKP99 @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Są np. Zielone paczkomaty: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Jeżeli tak ustalono z właścicielem terenu to nie ma się tutaj nad czym doktoryzować.: 0
@Wichura_ @hockeygoaliepl To Wojtek zaczyna. @PaczkomatyPL rozjebaly flaszke, ktora mu wyslalem i sie do mnie przypierdala🤭: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Paczkomaty gurom.: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Faktycznie ta brudna biedra bez tego paczkomatu była taka piękna :-D: -1
@PaczkomatyPL @Rafi__X A pracownicy to rozwożący są przynajmniej dobrze opłacani za pracę w weekendy?: 0
znowu @PaczkomatyPL ??? Chyba nie ogarniacie tematu, przerósł was "sukces"? https://t.co/iZjEmADWeF: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Na wsi ludzie są od dziesiątek lat tak traktowani. Paczkomat to co jak co nowy element małej architektury, który możliwie ze nieprędko zniknie - a in post zgarnąłby naprawdę sporo prasy gdyby ogłosił konkurs architektoniczny na wpasowanie swoich rozwiązań lepiej w architekturę. https://t.co/R9alDbp9WI: 0
@Muchoborek @MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Polska.: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ To samo dotyczy całodobowych myjni samochodowych pod oknami mieszkań! Co to jest do kurwy nędzy?: -2
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ To na Międzynarodowej? Czuj to teren?: 0
@MiastoJestNasze @PaczkomatyPL @warszawa @trzaskowski_ @molszew77 @Marlena_Happach @Kucharski_PO @pieinggg @JanMencwel @barb_jed @MartaMarczak_ Paczkomaty to naprawdę w założeniu fajny pomysł, jednak można to rozplanować lepiej.: 2
@Rafi__X @PaczkomatyPL Też pół godziny temu dostałem powiadomienie, że paczka do odbioru :): 0
@Rafi__X @PaczkomatyPL Tez dostałem dziś 👍👌: 0
./out/ee5d6d5c-3db7-4910-84ff-124588da9dd9.zip is empty! No sentiment for value 316
@Rafi__X @PaczkomatyPL Tak😀: no sentiment
@Rafi__X @PaczkomatyPL W okresie świątecznym tak co roku jest.: 0
Hej @PaczkomatyPL, to Wy teraz w niedzielę nawet dostarczacie? https://t.co/YX7CCSNCBc: 0
@PaczkomatyPL To chyba nie zawsze, moja paczka była wydana do doręczenia w piątek, a obecnie jest przyjęta z powrotem w oddziale drugi dzień xd https://t.co/PwZXDYfaij: 1
@PaczkomatyPL Tylko dlaczego nie dostarczył jak wszyscy byli w domu?: 0
@PaczkomatyPL To chyba ktoś nie zrozumiał pouczenia bo wszyscy w domu, żona widzi kuriera jak jedzie a paczka nie zostaje dostarczona i będzie ponowna próba.: 0
@bartbuks @CarlossDee @PaczkomatyPL @Allegro_Group Ja już wcześniej dostawałem w niedziele.: 0
@bartbuks @PaczkomatyPL @Allegro_Group Wczoraj wieczorem jedna mi wskoczyła, dziś rano druga.
Jedna z nich zamówiona w piątek 😁: 0
@CarlossDee @PaczkomatyPL @Allegro_Group No właśnie w szoku, że paczka doszła 😜: 0
Kurierzy @PaczkomatyPL nawet dziś działają 😱
Dają radę z @Allegro_Group 👍: 0
@PaczkomatyPL Zobaczcie swoje DM z piątku 10.12. Bo sami chcieliście wiadomość . Rozumiem, że ktoś inny obsługiwał konto ale czy coś się zadziało w tej sprawie? Czy jednak oficjalnie przez formularz?: 0
@pajkiel @PaczkomatyPL No to się ciesz, u mnie jest status przyjęta w oddziale od wczoraj...: 0
./out/6f822401-3e82-4877-abcb-3812ba6fb9d3.zip is empty! No sentiment for value 328
@krzyniu_ @PaczkomatyPL Hmm... https://t.co/TF2wd2olYR: no sentiment
@pajkiel @PaczkomatyPL Widocznie pozostałe paczkomaty też były zapełnione, jak to bywa przed świętami.: 1
./out/e3068b7e-14de-4ec1-beb2-07cc4b18099a.zip is empty! No sentiment for value 330
@krzyniu_ @PaczkomatyPL To już nie https://t.co/m6sCucI72Q: no sentiment
@pajkiel @PaczkomatyPL To co wkleiłeś dotyczy tylko 23 i 24 grudnia i gwarancji dostawy na święta, gdy przesyłka została nadana max 22 grudnia do godziny 14:00.: 0
./out/d4cf799f-a880-4c79-a7b5-0fb6a3e93bab.zip is empty! No sentiment for value 332
@krzyniu_ @PaczkomatyPL https://t.co/XyynIsudqX: no sentiment
@pajkiel @PaczkomatyPL Może jeszcze dotrze.: 0
@pajkiel @PaczkomatyPL W zeszłym roku było coś takiego jak przekierowanie do innego paczkomatu, natomiast trzeba było na to wyrazić zgodę. Teraz o to nie pytają więc nie mogą Ci w innym zostawić.: 3
@justyna560 @PaczkomatyPL Ja przez apke sprawdzam przeciez: 1
@pajkiel @PaczkomatyPL Nie wysyłają teraz sms-ów. Sprawdzaj w aplikacji.: 1
@krzyniu_ @PaczkomatyPL Na terenie mam kilka paczkomatow: 0
@pajkiel @PaczkomatyPL Obstawiam, że w paczkomacie miejsca nie było?: 1
./out/9425c729-1b2e-44a1-9490-570ba2658b30.zip is empty! No sentiment for value 339
@krzyniu_ @PaczkomatyPL Oszusci jacys: no sentiment
@pajkiel @PaczkomatyPL Wczoraj miałem taki sam komunikat, o 23 dostałem kolejny, że Przyjęta w oddziale Inpost xd: 0
I gdzie to doręczenie do paczkomatu? ja się pytam @PaczkomatyPL https://t.co/vg9GBKbohs: -1
./out/3b1c0fa2-207e-4149-811d-648d5d310f2c.zip is empty! No sentiment for value 342
@PaczkomatyPL 🙃: no sentiment
@PaczkomatyPL Ale to jest paczka do domu: 0
@PaczkomatyPL Dojedzie jeszcze dziś? :v https://t.co/v0IsVNF5YW: -1
@xkom_pl @PaczkomatyPL No i super, o ile InPost nie nawali xD: 1
@michaaaalll @PaczkomatyPL Wychodzi na to, że kurier się obraził za to, że mi zostawił paczkę z Xboxem u obcych ludzi, zwróciłem mu uwagę i mówiłem, że to zgłoszę a dwa dni później zrobił mi awanturę i dopiero wtedy zgłosiłem na DM. Mam nadzieje, że to przypadek a nie złośliwość bo wszyscy dziś w domu byli: 1
@vinted + @PaczkomatyPL to jest złoto! #elumping: 0
@michaaaalll @PaczkomatyPL A jak nie ma to trafia do punktu odbioru lub innego paczkomatu. Najlepsze, że żona mi teraz mówi, że widziała go z balkonu po 14 jak tu jechał a go nie było. Cały dzień w domu jestem a po 16 trafiła inna paczka do paczkomatu. Koleś mi ostatnio awanturę na korytarzu zrobił: -2
@michaaaalll @PaczkomatyPL Kurier do domu bo do paczkomatu inna paczka trafiła: 0
@LeszekJarema @PaczkomatyPL Może nie było miejsca w paczkomacie: 0
Wiecie co oznacza „kolejna próba doręczenia” w @PaczkomatyPL ? https://t.co/eLalBpt3ru: 1
@PaczkomatyPL Ale tym, że wpływacie na podświadomość klienta i sugerujecie, kiedy powinien wyjść z paczką na spacer, to się nie chwalicie 🤣: 0
Trzy minuty. 👌 Najwidoczniej trzecia dawka nie łączy z pralką, ale z @PaczkomatyPL 👌👌👌 https://t.co/GkTlqCbrXN: 0
RT @emocjewsieci: W szczycie świątecznego szału na paczki sprawdzamy, jak wygląda natężenie sentymentu w komentarzach o @PaczkomatyPL i @Po…: 2
W szczycie świątecznego szału na paczki sprawdzamy, jak wygląda natężenie sentymentu w komentarzach o @PaczkomatyPL i @PocztaPolska na portalach typu opineo, jakoscobslugi, ceneo, opiniuj, trustedshops, gowork czy shopify. Od 1 grudnia do wczoraj wygląda to tak. 👇 https://t.co/KUsinOXXNQ: 2
@PaczkomatyPL Szanujemy 🦾: 0
@kwiatekxson @PaczkomatyPL U nas wszystko zależy od konkretnego produktu. Na każdej podstronie informujemy jaki jest przybliżony czas dostawy produktu: 3
Hej @PaczkomatyPL a czy dostarczycie list do Świętego Mikołaja przed świętami? Córka pyta.: 1
Jezu paczka o 12 wydana wczoraj do doręczenia i dalej jej nie ma… @PaczkomatyPL @InPostPL1 co się dzieje 😡: 0
./out/cf35c068-4e4f-49a1-bae6-48fca41740bb.zip is empty! No sentiment for value 360
@PaczkomatyPL Ok: no sentiment
@PaczkomatyPL Aha i nie wiem jak to się dzieje, ale ludzie NIE czytają, że paczka trafiła do innego APM: 0
@PaczkomatyPL Doczekam się odpowiedzi ?: 1
Wybieranie prezentów do ostatniej chwili? Prawie! Przesyłki nadane do 22 grudnia trafią do adresata przed pierwszą gwiazdką! Taką gwarancję daje tylko #InPost, który jest w pełni gotowy na świąteczny gorący okres. @PaczkomatyPL @RBrzoska #Święta #prezenty https://t.co/Sv03AdgsDf: 1
Tydzień przed świetami firma kurierska nie pracuje. XD Brawo @UPSPolska, dobrze, że większość paczek zamawiam przez @PaczkomatyPL bo inaczej bym się nie doczekał dostawy paczek.: -1
@PaczkomatyPL Stawiajcie APM w dostępnych miejscach dla kurierów i serwisu.
W aplikacji dodajcie info, że paczkomat jest aktualnie niedostępny.: -1
@Improbite @PaczkomatyPL Albo paczka jest uszkodzona :)))) Taki przypadek miałam ostatnio z @galaxy_mutant i wyobraź sobie, że gdy reklamacja została złożona to do tej pory cisza, żadnej odpowiedzi... Ale mam nadzieję że w Twoim przypadku to tylko przepełniony paczkomat: 0
@Harrolfo @PaczkomatyPL To bardzo ważna zmiana!: 1
@PaczkomatyPL Udoskonalić aplikację mobilną, żeby można było do niej przydzielić kilka numerów a nie tylko jeden. Przydatne gdy ktoś ma numer służbowy i prywatny i odbiera paczki na obydwa numery.: 1
@PaczkomatyPL Proszę o wyjaśnienie , dlaczego paczka nie trafiła do wybranego paczkomatu. To jest treść dzisiejszego maila od Was.

Czas na odbiór paczki magazynowanej w PaczkoPunkcie upłynął.

Nic straconego! Paczka od Amazon EU SARL wkrótce zostanie dostarczona do pierwotnie wybranego Pacz: 2
@PaczkomatyPL Paczka miała zostać dostarczona do paczkomatu , ponieważ nie było miejsca w paczkomacie paczka trafiła do paczkopunktu , następnie miała trafić do wybranego paczkomatu. Pierwszy raz doświadczam problemu z paczkomatem.: 0
@PaczkomatyPL Moja przesyłka z wróciła do oddziału , kiedy zostanie dostarczona do paczkomatu ?

653999410056400004160664: 0
@RzecznikOrange @PaczkomatyPL @InPostPL1 W gratisie darmowe możliwe incydenty naruszenia ochrony danych osobowych, jak w przypadku dostawy kurierem, czy tam jednak bardziej dbacie o swoich klientów? Pytanie całkiem serio panie Wojciechu.: -2
@PaczkomatyPL mikołaj nie odpisał w którym paczkomacie zostawił walizkę piniedzy dla mnie: 1
./out/beb69252-c85a-4b0f-ad17-9bb6d04c3c8b.zip is empty! No sentiment for value 374
@fijolekmagda @PaczkomatyPL "Never have been": no sentiment
@ja87790041 @PaczkomatyPL U siebie nie mam problemów. Jeśli w jednym się zerwie, to zmieniam w polu, a jeśli w drugim się zerwie, to zmieniam silnik :D: 0
@fijolekmagda @nazwa_zajeta @PaczkomatyPL @galopujacymajor ale oni tam muszą mieć słaby wskaźnik rotacji zapasów.: 0
@fijolekmagda @nazwa_zajeta @PaczkomatyPL @galopujacymajor Pełna zgoda! Na przykład dzisiaj poszedłem na pocztę i była zamknięta, ale wracając zobaczyłem źle zaparkowane samochody i wykonałem telefon do Straży Miejskiej. Gdyby nie poczta, to nadal by tam stały! #TeamPocztaPolska: 0
RT @fijolekmagda: @nazwa_zajeta @PaczkomatyPL @galopujacymajor Może mnie cancelować, ale ja jestem #TeamPocztaPolska, z której właśnie serd…: 0
@TPBlaszczyk @CywilizacjaZ @PaczkomatyPL Wg mojej najlepszej wiedzy nie. :/: 1
@PaczkomatyPL No spoko byczq właśnie idę odebrać i też mam nadzieję, że skończy się happy endem 🥰: 4
@nazwa_zajeta @PaczkomatyPL @galopujacymajor Poza tym mam świadków na to, że Poczta rlz 🥰 cc: @kepinski_razem: -2
@nazwa_zajeta @PaczkomatyPL @galopujacymajor Może mnie cancelować, ale ja jestem #TeamPocztaPolska, z której właśnie serdecznie pozdrawiam xD https://t.co/bbJUwYQ1vc: 0
@psztymucel @CywilizacjaZ @PaczkomatyPL Nie macie jakiejś apki miejskiej do obywatelskiego podpierdalania partyzantki parkingowej?: 0
@mrowinski_pawel @nazwa_zajeta @fijolekmagda @PaczkomatyPL @galopujacymajor Po prostu kolejki są wtedy dłuższe 🙄: 0
@nazwa_zajeta @fijolekmagda @PaczkomatyPL @galopujacymajor Poczta nigdy nie jest przepełniona 🤷‍♂️: 0
@CywilizacjaZ @PaczkomatyPL Zaraz tam łamać. Parkowanie na chodniku generalnie jest legalne, a wszystkimi czterema kołami i za pasem zieleni najwyraźniej zdekryminalizowane, bo przecież inaczej choć czasem ktoś by to ścigał. ;&gt;: -1
@CywilizacjaZ @PaczkomatyPL Od stycznia taryfa za wjazd na chodnik będzie 1500. I jestem przekonany, że aktywiści i inni widzowie będą fotografować, filmować, zgłaszać. Kurierzy szybko przeproszą się z wózkami.: 0
@fijolekmagda @PaczkomatyPL czekaj aż się @galopujacymajor dowie: 0
Dostałam informację od @PaczkomatyPL, że mój paczkomat jest "przepełniony" i w związku z tym moja przesyłkę pozostawiono w innym. Nadal jesteście przeciwko depopulacji? 😠: 1
@psztymucel @PaczkomatyPL Ma prawo sądzić, że skoro wolno mu postawić paczkomat, to i łamać przepisy obsługując go? :-): 0
@CywilizacjaZ @PaczkomatyPL No ale ma pełne prawo sądzić, że skoro instytucja powołana do utrzymania drogi w stanie używalności mu na to pozwala, to jest taki znów wielki problem.: 1
./out/7c71e43c-b503-4329-84b6-7c8e8def35ff.zip is empty! No sentiment for value 392
@jedrek @psztymucel @PaczkomatyPL Tak, to również.: no sentiment
@psztymucel @CywilizacjaZ @PaczkomatyPL Często zarządca nie odpowiada za teren "postojowy" przy paczkomacie, tylko za samą przestrzeń na której stoi paczkomat.: 0
@psztymucel @PaczkomatyPL Bo paczkomaty na tym zarabiają. Oczywistym jest, że każdy biznes posunie się tak daleko, jak mu pozwoli dana administracja, ale to nie zmienia faktów: InPost zarabia rozjeżdżając chodniki.: 0
@CywilizacjaZ @PaczkomatyPL Mają na to wy*bane. Co to, ich problem?: 0
@CywilizacjaZ @PaczkomatyPL Ale czemu Paczkomaty ma o to dbać, a nie zarządca drogi, który na to pozwala?: 0
./out/dd6c5d79-9cb7-4274-9d05-d3acb6325882.zip is empty! No sentiment for value 397
@CywilizacjaZ @PaczkomatyPL Tak: no sentiment
@PaczkomatyPL tak jak Was zawsze zachwalałem, tak dziś jestem Wami rozczarowany. W środę zamówiłem prezent urodzinowy (które są dziś) i zawsze następnego dnia paczka wychodziła a potem była u mnie… dziś jak na złość paczka od 4tej rano wciąż tkwi w oddziale…nie jest na wydaniu.: 0
RT @CywilizacjaZ: Czy @PaczkomatyPL montując paczkomaty w głębi chodnika, bez miejsca postojowego dla samochodu kurierskiego wie, że to ozn…: -1
./out/6854dc69-0a41-4059-abbf-125bb0fc1f3f.zip is empty! No sentiment for value 400
@CywilizacjaZ @PaczkomatyPL Tak.: no sentiment
Czy @PaczkomatyPL montując paczkomaty w głębi chodnika, bez miejsca postojowego dla samochodu kurierskiego wie, że to oznacza wjeżdżanie samochodem na chodnik, między ludzi, i po prostu o to nie dba?: -8
@PaczkomatyPL Czy istnieje szansa na możliwe nadawane w przyszłości przesyłek międzynarodowych przez Paczkomaty? z Polski? :): 1
@PaczkomatyPL Mój ojczym pracuje u was i jeżdżę z nim do pracy, tylko jest problem ze niektóre paczkomaty mają trudny dojazd: -1
@RBrzoska @JanSpiewak @PaczkomatyPL No i fajnie👍brawo za szybka reakcje👏 wesolych swiat☃️☃️☃️: 1
RT @xkom_pl: @PaczkomatyPL Hipotetycznie: jeśli nasz kolega utknął w dużej skrytce w paczkomacie, to (hipotetycznie) ile będzie w stanie ta…: 1
@PaczkomatyPL Przed świętami dostarczacie paczki max do,22.12.2021?! @PaczkomatyPL @xkom_pl  a ostatni dzień na wysyłkę przed świętami u was to do kiedy?! Aby inpostem dotarła ?!: 0
Co jak co, ale opcja umieszczenia paczki w dolnej skrytce jest świetna. Sam co prawda z tego nie korzystam (mam 185 cm wzrostu), ale brawo dla @PaczkomatyPL za pomyślenie o niższych osobach.

Wiem, że ta opcja działa już od pewnego czasu, ale znam osoby, które o tym nie wiedzą 😉 https://t.co/u5f47Xvqgq: -1
@PaczkomatyPL Hipotetycznie: jeśli nasz kolega utknął w dużej skrytce w paczkomacie, to (hipotetycznie) ile będzie w stanie tam przetrwać bez swojego PC? 🤔: 1
@PaczkomatyPL Jestem sklepem z zabawkami, sprzętem AGD małym i dużym, sprzętem ogrodowym i fitness. Wysyłam teraz mnóstwo prezentów na Święta. 
Pomagacie na co dzień, dziękuję za super współpracę💙💜: 1
@ PaczkomatyPL a moze szybciej https://t.co/vtCXQmlg24: 0
@PaczkomatyPL Co zrobić na obiad? Carbonara, czy tortilla z kurczakiem?: -1
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
@SzymonSorokiew @PatrykSlowik @PaczkomatyPL Małe myki? To nie są żadne myki, ani podkręcanie tematu. Skoro miejsce paczkomatu było „kretyńskie”, to naszą rolą jest to pokazywać i o tym mówić. Gdyby nie media, ten paczkomat nadal by tam stał, bo przecież „jest też druga brama”. Tyle.: 0
@Improbite @PaczkomatyPL Pewnie się nie zmieściła w docelowym, ani w żadnym pobliskim paczkomacie. Są święta, jak zawsze paczkomaty są zapchane ;]: 0
@Improbite @PaczkomatyPL Co Ty gadasz. Fajna lipa: 1
@Improbite @PaczkomatyPL Paczka podróżująca w czasie?: 0
Hej @PaczkomatyPL, dlaczego paczka, która miała status „wydana do doręczenia” wczoraj, dziś ma „przyjęte w oddziale”?  😒: 0
@k_michalec @PatrykSlowik @PaczkomatyPL Umiejscowienie paczkomatu było kretyńskie. Ale zawsze bawi mnie dodatkowe podkręcanie atmosfery za pomocą małych myków.: 1
@SzymonSorokiew @PatrykSlowik @PaczkomatyPL A co to zmienia? Rozumiem, że jak z dwóch wąskich przejść zostaje jedno, bo drugie jest zastawione paczkomatem to nie ma problemu? Naprawdę nie popadajmy w absurd: 0
@PaczkomatyPL Dałoby radę zrobić tak żeby wszystkie listy przewozowe jak te wystawiane przez Amazon nie miały nadrukowanymi danych osobowych, maila i telefonu?: 2
@PaczkomatyPL Zerwał mi się pasek rozrządu w aucie, znacie się może na mechanice?: 0
@PaczkomatyPL hej, niestety Mat nie działa. Jest jakiś sposób aby skontaktować  z żywym agentem?: -2
@k_michalec @PatrykSlowik @PaczkomatyPL Proszę Pana, dokładnie TAKIE samo ujęcie pokazane jest wcześniej RAZ. Przez maks. sekundę. 
Poza tym są jeszcze 2 ujęcia. Pierwsze jest pod kątem i w ruchu przez co nawet nie widać, że tam jest brama. 
Kolejne  ujęcie to wąski kadr 2. bramy i nie widać odległości do 1-ej. https://t.co/mgTyY1uEGv: -2
@k_michalec @PatrykSlowik @PaczkomatyPL Zaraz sobie obejrzę jeszcze raz...: 0
@SzymonSorokiew @k_michalec @PaczkomatyPL To w praktyce niemożliwe, bo dopiero analiza gruntu pozwala na określenie dokładnego miejsca. A InPost nie chce jej przeprowadzać, dopóki nie ma umowy.: 0
@PatrykSlowik @k_michalec @PaczkomatyPL Od spółdzielni/wspólnoty oczekiwałby jednak, że wydają zgodę na BARDZO konkretną lokalizację. 
Poza tym wypowiedzi tej strony zdecydowanie brakowało w materiale.: -2
@SzymonSorokiew @PatrykSlowik @PaczkomatyPL W materiale jest to ujęcie i podobne użyte kilka razy: -1
@PatrykSlowik @k_michalec @PaczkomatyPL Nie bronię @PaczkomatyPL bo to beznadziejna miejscówka,  ale warto zauważyć, że 2 m dalej jest takie samo przejście, a pokazano to na samym końcu materiału przez ok 2 sekundy. Motywacja ta sama co dla Śpiewaka: zdjęcie z szerszym kadrem nie robiłoby takich zasięgów w mediach. https://t.co/bKU7S2uMSv: 0
@SzymonSorokiew @k_michalec @PaczkomatyPL Ze zgodami bywa różnie. Często zgoda jest na paczkomat i teren (działkę), ale bez wskazywania konkretnej lokalizacji.
Wiem, bo sam ostatnio wydawałem zgodę na postawienie paczkomatów. I postawiono głupio.: 12
@PatrykSlowik @k_michalec @PaczkomatyPL InPost siłą tam tego paczkomatu nie postawił. Wspólnota wydała na to zgodę. Szkoda, że dziennikarze nie wysilił się, aby porozmawiać z ich przedstawicielami, bo jestem ciekaw co mieliby do powiedzenia na temat genialnej decyzji.: 0
./out/f1544912-a2f4-474c-a07d-8b107a284d55.zip is empty! No sentiment for value 431
@PatrykSlowik @PaczkomatyPL Dzięki ;): no sentiment
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
Żółty #Mikołaj? 💛🎅 Ten na pewno dotrze na #Święta! Oczywiście z #InPost, dla którego gwarancja dostawy to już tradycja. 2 lata temu zapewniał dostawę przesyłek nadanych do 20, rok temu do 21, w tym roku nawet do 22 grudnia! 🎁 @PaczkomatyPL @RBrzoska https://t.co/AdL09FRpta: 0
Świetny materiał @k_michalec.
Oburzyła mnie wypowiedź rzecznika @PaczkomatyPL. Paczkomat - oficjalnie - usunięto nie dlatego, że postawiono go w patologiczny sposób i ograniczono ludziom poruszanie się, lecz z powodu znalezienia atrakcyjniejszego miejsca.
https://t.co/Zp05gIjVMh: 0
@WKadziolka @k_michalec @PaczkomatyPL @wirtualnapolska @JanSpiewak Tak, jest o tym wzmianka w materiale ;): 0
Kolejna, godna pochwały ekoinicjatywa @PaczkomatyPL @RBrzoska mająca na celu ogromną redukcję zapotrzebowania na jednorazowe opakowania przesyłek. W akcji bierze już udział sklep Modivo z @eobuwie_pl @GrupaCCC #goz #odpady #recykling #paczkomaty
https://t.co/92r4vB6nQb: 0
Panie Brzoska- co za głupota!@PaczkomatyPL https://t.co/E8zJKbj83t: 0
Szyba podpowiedź rzecznika. Od dziś do niedzieli 19 grudnia, trwa nasza akcja darmowej dostawy do @PaczkomatyPL @InPostPL1. Dotyczy klientów indywidualnych i biznesowych z e-umową, w wybranych ofertach. Warto skorzystać.: 0
@missysmarties @JanSpiewak Niczyją intencją jest tworzenie utrudnień nawet w rozwiązaniach tymczasowych-@PaczkomatyPL nie tylko wyciągają wnioski ale tez reagują na głosy z każdej strony.Wesołych Świąt dla wszystkich zainteresowanych 🎄🎄🎄: -2
@rav22wawa @MiastoJestNasze @PaczkomatyPL To nie wygląda na wejście do klatki. Nie znam tego miejsca ale znam sporo takich domów i wejścia są zwykle od podwórza. To prędzej jakieś gosp.
Jest też na pewno jakiś wjazd, takie przejścia to skróty dla pieszych.: 2
@SialaTV @k_michalec @PaczkomatyPL @wirtualnapolska @JanSpiewak Paczkomat został postawiony na prośbę wspólnoty, która wskazała nam tą lokalizację. Od początku traktowaliśmy tę lokalizację jako tymczasową. Ponieważ naleźliśmy lepsze miejsce w pobliżu Paczkomat został już relokowany. Wojciech Kądziołka, rzecznik prasowy InPost: 0
@WojtekKardys @PaczkomatyPL @InPostPL1 Paczkomat został postawiony na prośbę wspólnoty, która wskazała nam tą lokalizację. Od początku traktowaliśmy tę lokalizację jako tymczasową. Ponieważ naleźliśmy lepsze miejsce w pobliżu Paczkomat został już relokowany. Wojciech Kądziołka, rzecznik prasowy InPost: 0
Taki SMS dostałem dzisiaj. Jeśli dostaliście podobny to radzę nie klikać w link. Ja jestem na to wyczulony i nie klikam. 
Czy @InPostPL1 @PaczkomatyPL zna ten problem? https://t.co/nEn5F5bc9C: 0
RT @SialaTV: Brawurowy materiał @k_michalec o @PaczkomatyPL na Moko :) jest też rozwiązanie zagadki - oglądajcie na @wirtualnapolska :) 
@J…: 0
@WojtekKardys @PaczkomatyPL @InPostPL1 To fotomontaż bie wierze: 0
Jak jestem fanem maszyn paczkowych, to ten przypadek pokazuje, że @RBrzoska powinien zatrudnić w tym #InPost kogoś, kto będzie pierw weryfikował udostępnione lokalizacje pod  @PaczkomatyPL 🤯 A może już jest takie stanowisko i ktoś właśnie robi wylot? 🧐. Gdzie wysłać CV? 😏 https://t.co/eJAoycsAWY: -1
RT @marta_sofie: @PaczkomatyPL @InPostPL1 Szanowni ...od trzech dni nie potrafią wyjąć paczki z zepsutego paczkomatu
Pytani o odszkodowanie…: 0
RT @SialaTV: Brawurowy materiał @k_michalec o @PaczkomatyPL na Moko :) jest też rozwiązanie zagadki - oglądajcie na @wirtualnapolska :) 
@J…: 0
Brawurowy materiał @k_michalec o @PaczkomatyPL na Moko :) jest też rozwiązanie zagadki - oglądajcie na @wirtualnapolska :) 
@JanSpiewak 
https://t.co/YLLiANtRCw: 0
@RBrzoska @pbzatonski @PaczkomatyPL Mozna powiedziec ze ten paczkomat zdobi ten ochydny budynek: 1
@marta_sofie @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Podaj mi ma DM dane paczkomatu to jutro postaram się uzyskać numer.: 0
@Piomek18 @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Nie podali bo twierdzą, że to problem techniczny a nie kurier: 0
@rav22wawa @MiastoJestNasze @PaczkomatyPL Obok jest drugie rownolegle przejście. Ale psułoby rwetes śpiewakom.: -2
@WojtekKardys @PaczkomatyPL @InPostPL1 Ah te pisanie aby pisać to takie typowe polskie. Jak widać odczytanie artykułu trudniejsze niż obejrzenie zdjęcia :): 3
@Anchel005 @rav22wawa @MiastoJestNasze @PaczkomatyPL Karetka tam nie musiala by wjezdzac, wystarczy zeby zespol ratunkowy zmiescil sie na piechote.: 0
@rav22wawa @MiastoJestNasze @PaczkomatyPL Wg Ciebie tam wcześniej zmieściłaby się karetka?: 0
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
Czego jak czego, ale doręczenia paczki do @PaczkomatyPL o tej porze to się nie spodziewałem: 0
@marta_sofie @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Na infolinii inpost mają numer do kuriera który obsługuje dany obszar.: 0
@Piomek18 @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Nie ma kontaktu
Gdzie masz kontakt do kuriera z Inpostu? Nie potrafią otworzyć: -1
@marta_sofie @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Może źle zabrałaś się do tematu.

Ja najpierw skontaktowałbym się z kurierem by otworzył paczkomat.: -1
@Piomek18 @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Za przetrzymywanie a tym samym narażenie na zniszczenie przesyłki: 0
@marta_sofie @PaczkomatyPL @InPostPL1 @ForfiterP @AMourning2 Odszkodowanie za awarię?: -1
RT @marta_sofie: @PaczkomatyPL @InPostPL1 Szanowni ...od trzech dni nie potrafią wyjąć paczki z zepsutego paczkomatu
Pytani o odszkodowanie…: 0
@PaczkomatyPL @InPostPL1 Szanowni ...od trzech dni nie potrafią wyjąć paczki z zepsutego paczkomatu
Pytani o odszkodowanie twierdzą, że nie ma podstaw
#helpunku
@ForfiterP @AMourning2 @Piomek18 https://t.co/HMy519mLNF: 0
@yola_nda13 @Exen @PaczkomatyPL no z fortepianem już nie przejdziesz 😉: 0
@Exen Hej @PaczkomatyPL  gdzie  to cudo stoi? I jak profesjonalnie wypoziomowane.: 0
@WojtekKardys @PaczkomatyPL @InPostPL1 Sprawny deweloper wcisnąłby tam  jeszcze ze trzy apartamenty.: 1
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
@pbzatonski @PaczkomatyPL Dokładnie tak.lokalizacja tymczasowa na bazie prośby wspólnoty ale MJN jak widać „grzeje temat” dziwnym trafem zawsze pomijając automaty innych firm np podłączonych do latarni miejskiej albo ustawionych w podobnych miejscach-zastanawiające prawda?: 2
@Exen Współczuję dla kuriera @InPostPL1 @PaczkomatyPL 😂: 0
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
RT @WojtekKardys: Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wsty…: 0
Ale ten paczkomat w bramie to jest taka typowa Polska. 🤦🏻‍♂️

Hej @PaczkomatyPL i @InPostPL1 - podziałajcie z tym bo wstyd. https://t.co/JxN7mi4jS8: -4
./out/8ec7a17d-9043-4dc4-91ef-388e8c39ed7a.zip is empty! No sentiment for value 476
@PaczkomatyPL @modivo https://t.co/6PSDnhhRBs: no sentiment
Nie wygląda to na ustawienie docelowe, ale może @PaczkomatyPL @RBrzoska się odniosą? https://t.co/leDu3InHuR: 0
RT @WZylm: Może kiedyś paczka się znajdzie…🤷‍♂️ @PaczkomatyPL https://t.co/jtA6KiyB6O: -1
RT @PaczkomatyPL: Pierwszy w Polsce system wtórnego obiegu opakowań w e-commerce staje się faktem! Wspólnie z marką @Modivo testowo wdrażam…: 1
@rotax73 @PaczkomatyPL @InPostPL1 😊😊
Dobrze, że w przesyłce nie było tego od czego jest pudełko a aceton był w butelce plastikowej. Urządzenie elektroniczne by tego nie przetrwało.: 0
@efka_k_ @PaczkomatyPL @InPostPL1 https://t.co/nf8uKaiFQm: 0
Brawo @PaczkomatyPL trochę podparte, pogniecione, mokre (zawartość się ulała) ale jest, paczka dostarczona. @InPostPL1 https://t.co/tPPdriWnQa: 0
Jaka powinna być #logistyka? Przy obecnym wzroście #ecommerce - nowoczesna, wydajna i ekologiczna. #InPost @PaczkomatyPL dba o środowisko, stąd pomysł na #opakowaniazwrotne. To przyszłość handlu. #ekologia #CCC #modivo @NowyMarketing @RBrzoska  https://t.co/RnvyPHynjR: 2
Pierwszy w Polsce system wtórnego obiegu opakowań w e-commerce staje się faktem! Wspólnie z marką @Modivo testowo wdrażamy opakowania wielokrotnego użytku, które będzie można zwrócić w dowolnym Paczkomacie. Więcej informacji: https://t.co/76sT2z2KZf https://t.co/amhzC3vCSF: 1
hej @PaczkomatyPL, gdzie można skontaktować się w sprawie zaginięcia/ciągłego wożenia paczki w samochodzie? od 2 dni w doręczeniu, inne paczki przychodzą normalnie, a ta jedna gdzieś zniknęła:(: -2
@rav22wawa @MiastoJestNasze @PaczkomatyPL To też zasługa wspólnoty: 0
@MiastoJestNasze Ciekawe jak z tych schodów wyjdzie ktoś z wózkiem, jak wyjdą ratownicy medyczni z noszami, jak wynieść meble ... Czy wy @PaczkomatyPL macie jakiś mózg?: 3
@PaczkomatyPL paczucha już jest u mnie 🥰 dziękuję!: 0
@MiastoJestNasze Naprawdę @PaczkomatyPL ? Serio postawiliście paczkimat w PRZEJŚCIU?: 1
@PaczkomatyPL Czy juz coś wiadomo co z reklamacją?: 0
@AmazonHelp Przejdę się i odbiorę. Dzięki za zainteresowanie :) Warto jednak poprawić komunikat w Waszym systemie, bo obecny wprowadza klienta w błąd. Paczka nie znajduje się w wybranym punkcie (@PaczkomatyPL), tylko w PaczkoPunkcie https://t.co/d1ZQr9ewgW: -1
@PawelLenarczyk @AllegroGroupEU Paweł, Inpost &lt;&gt; Allegro. Raczej @PaczkomatyPL bym oznaczał, @rbrzoska nawet.: 1
Hej @PaczkomatyPL , o co tutaj chodzi? https://t.co/GVNTwc96Fp: 0
@Pawel__Bieniek @PaczkomatyPL Widać, że ktoś musi mieć gorzej, aby ktoś miał lepiej :p Nie chcę zapeszać, ale u mnie chodzą przesyłki jak w zegarku. User experience - mega. Zupełnie inaczej  niż kursy Inpostu i Allegro :): -4
@GazetaStoleczna Tak na marginesie zaznaczę, że nie znam lepszej opcji niż @PaczkomatyPL.  Nic nie muszę wypisywać, drukować, wszystko ogarniam w apce. Wkładam przesyłkę do paczkomatu, a ta - w okresie przedświątecznym - dociera do odbiorcy w mniej niż 24 godziny. https://t.co/2SsLwTFlym: -1
@pocoto_wszystko @PaczkomatyPL A teraz mówi, że marźnie... żeby odebrać. Do ostatniej chwili będę czekał. Nie będę specjalnie leciał, żeby zwolnić miejsce.: 0
@Lurker_Land @InPostPL1 A nie do @paczkomatypl? :P: 0
Paczkomaty są wygodne. Zostaną z nami na dłużej. Kiedyś pewnie przeminą, jak kioski prasowe. Uważam, że warto naciskać na @InPostPL1, aby @PaczkomatyPL stawiane były z głową, ładem i ZWSem, żeby też były po prostu ładniejsze, choć niekoniecznie niczym pałacyki wody oligoceńskiej. https://t.co/HqwmrCW1Qy: -2
@PaczkomatyPL Pomyślałem, pomyślałem i mam pytanie. Czy indywidualna etykieta opakowania zastąpi etykietę adresową? Przecież wystarczy skojarzyć w systemie kod QR z bieżącym nadawcą i adresatem! Czy tak to jest pomyślane? To byłaby kolejna oszczędność papieru oraz tuszu/toneru w drukarkach!: 1
@PaczkomatyPL chcialabym się dowiedzieć jak długo paczka będzie jeszcze przebywać w sortowni?😉: 0
@WZylm @PaczkomatyPL No co no musisz być szybszy od kuriera następnym razem 👀: 0
@RioT_PL @PaczkomatyPL @pieinggg Ociehui xdddd: 0
@WZylm @PaczkomatyPL Ja nadal czekam 🥰 https://t.co/H0X0CU0qNV: 0
@WZylm @PaczkomatyPL Trzymam kciuki za pozytywny finał sprawy: 0
@D_Wielogorski @PaczkomatyPL Mają 30 dni na rozpatrzenie mojej reklamacji 🤷‍♂️: 0
@WZylm @PaczkomatyPL Ciekawe co w przypadku, kiedy się nie znajdzie: 0
./out/d1bdf54a-c03b-410a-a373-32f7c93173bf.zip is empty! No sentiment for value 507
@WZylm @PaczkomatyPL Padła -paczka***: no sentiment
@WZylm @PaczkomatyPL Pamietam jak paczka wypadła z samochodu w miejscu gdzie miała być włożona.Kobieta znalazła ta paczkę w kałuży i zadzwoniła ze kurierowi wypadła https://t.co/gdSgVbyrtNła zawartość była mokra :)) a oznaczyli mi ze paczka została włożona do paczkomatu: -1
./out/5066540e-503a-4c72-8759-32fda4135cd5.zip is empty! No sentiment for value 509
@Natanieluz @PaczkomatyPL https://t.co/AeXFiuAa2N: no sentiment
@D_Wielogorski @PaczkomatyPL Problem jest taki, że sam Inpost do mnie pisał, że paczkę zgubili i jej szukają :D: 1
@WZylm @PaczkomatyPL Miałem podobną sytuację, ale w śledzeniu było "doręczono o 23:59". Dziesiątki telefonów, non stop przysłowiowe trucie dupy i nagle się znalazła. Trzymam kciuki aby było podobnie: -2
RT @WZylm: Może kiedyś paczka się znajdzie…🤷‍♂️ @PaczkomatyPL https://t.co/jtA6KiyB6O: -1
@WZylm @PaczkomatyPL Może kurierowi spodobała się zawartość? 👀: 0
Może kiedyś paczka się znajdzie…🤷‍♂️ @PaczkomatyPL https://t.co/jtA6KiyB6O: -1
@Pawel__Bieniek @PaczkomatyPL Every teardrop is a waterfall na kursie ....: 0
@PaczkomatyPL Zgłosiłem sprawę na infolinii. Teraz czekam... do 48h na kontakt oddziału :D: 0
@Devid77777777 @PaczkomatyPL Podobnie do jakości usług przed świętami :): 1
./out/a3741dc5-b5af-4531-b9f1-619775535b34.zip is empty! No sentiment for value 518
@margincall___ @Marek2137 @KulagaWojciech @PaczkomatyPL https://t.co/8P2eqyzWJ4: no sentiment
RT @pieinggg: Ktoś na wózku, rodzice z dziećmi, ktoś o kulach, z walizką - muszą być ZACHWYCENI.

No @PaczkomatyPL  tego się po Was nie spo…: 2
./out/c6ccdec4-cefa-4734-bf4f-76f5e3abb30a.zip is empty! No sentiment for value 520
@michaaaalll @Marek2137 @KulagaWojciech @PaczkomatyPL https://t.co/8P2eqyzWJ4: no sentiment
@Pawel__Bieniek @PaczkomatyPL z ciekawości zapytam jak kształtuje sie kurs InPost po IPO ?: 0
@PaczkomatyPL No trudno :( 
Dziękuję za odpowiedź &lt;3: 0
@pocoto_wszystko To chyba jest z tym związane. A przejeżdżałem obok szafy, internet włączony... "perfect timing" @PaczkomatyPL: 2
@PaczkomatyPL jest opcja żebym sie dowiedziała bardziej dokładnie kiedy będzie moja paczka jak mam status "wydana do doręczenia" od 8h, bo tak no spieszy mi się mocno?: 3
@PaczkomatyPL "Sława, sława, sława... Wybrany przez Ciebie Paczkomat jest tak popularny, że czasem wszystkie skrytki są zajęte.
Dlatego paczka od *** czeka teraz na Ciebie w PaczkoPunkcie" i ...

Jesteście beznadziejni !!!: 1
@PaczkomatyPL @genesispr_ "Sława, sława, sława... Wybrany przez Ciebie Paczkomat jest tak popularny, że czasem wszystkie skrytki są zajęte.
Dlatego paczka od *** czeka teraz na Ciebie w PaczkoPunkcie" i ...

Jesteście beznadziejni !!!: 1
@PaczkomatyPL "Sława, sława, sława... Wybrany przez Ciebie Paczkomat jest tak popularny, że czasem wszystkie skrytki są zajęte.
Dlatego paczka od *** czeka teraz na Ciebie w PaczkoPunkcie" i ...

Jesteście beznadziejni !!!: 1
@Obserwator2222 @JanSpiewak @restory244 @PaczkomatyPL a ratownicy z noszami i chorym? a strazacy z wyposazeniem? itd...: 0
./out/a000b6dd-4822-45ef-9db4-63af0e6fe3f3.zip is empty! No sentiment for value 529
@PaczkomatyPL @pieinggg https://t.co/IREtHDzNHx: no sentiment
@Marek2137 @KulagaWojciech @pieinggg @PaczkomatyPL I bardzo dobrze ze utrudnia. Nie po to jest taśma żeby tam przechodzić.: 0
RT @pieinggg: Ktoś na wózku, rodzice z dziećmi, ktoś o kulach, z walizką - muszą być ZACHWYCENI.

No @PaczkomatyPL  tego się po Was nie spo…: 2
@Pawel__Bieniek @PaczkomatyPL taki czas :) obecnie zamawia się kuriera i to na adres firmy. te mają 'wyższy priorytet'  :): 3
Paczka wysłana 5.12 Status na 15.12: w sortowni... @PaczkomatyPL poważnie?: 1
@pieinggg @PaczkomatyPL A obok tego paczkomatu parking zrobimy jest tak, że przy miejscu parkingowym dla OzN jest zawsze kałuża jak napada. Zrobię zdjęcie i pokaże wam, bo ostatnio nieziemsko się na to wkurzyłam. (Jestem tam czasami, bo obok jest super apteka.): -1
./out/96b36721-a551-4286-97e8-af372c825d6d.zip is empty! No sentiment for value 535
@PaczkomatyPL @pieinggg XDDDDDDD: no sentiment
RT @pieinggg: Ktoś na wózku, rodzice z dziećmi, ktoś o kulach, z walizką - muszą być ZACHWYCENI.

No @PaczkomatyPL  tego się po Was nie spo…: 2
@pieinggg @PaczkomatyPL Jakaś firma ma gdzieś potrzeby OzN part2020203030 

A w moim mieście rodzinnym Paczkomat stoi tak, że ledwo mieszczę się z wózkiem na chodniku.: -2
Do kiedy można nadać paczkę, by ta trafiła do odbiorcy przed Świętami? @PaczkomatyPL znowu zaskakują &lt;3

https://t.co/LeJp9trepW: 1
@PaczkomatyPL Po skontaktowaniu się ze sprzedającym na Allegro, dlaczego nie wysłał wczoraj paczki odpowiedział, że kurier powiedział, że ma dużo paczek i nie przyjedzie. Rozumienie ten paradoks? 🤦‍♂️: 0
Kurierzy @PaczkomatyPL apelują do klientów o szybkie odbieranie paczek z paczkomatu. Ja apelują do kurierów, aby nie tłumaczyli się duża ilością paczek i odbierali paczki od sprzedających!: -3
./out/9dd094e1-bcb7-47be-b46c-ca973e78b440.zip is empty! No sentiment for value 541
@PaczkomatyPL https://t.co/VFNX1LEMAJ: no sentiment
@PaczkomatyPL DACIE RADĘ! MUSICIE! 😅: 0
@JakubRosik @PaczkomatyPL Miejmy nadzieję że pójdzie szybciej niż z pralką.: 1
@JakubRosik @PaczkomatyPL Ja w sobotę 9 odbierałem 😉: 0
Zamówiłem 8 paczek w okresie przedświątecznym. Proszę przeprosić ode mnie kurierów @PaczkomatyPL: 0
@PaczkomatyPL  i Modivo, które jest modową platformą e-commerce z @GrupaCCC, wprowadzają opakowania wielokrotnego użytku. Jest to działanie testowe, ale wkrótce wszystkie e-sklepy współpracujące z firmą Rafała Brzoski będą mogły z nich korzystać. @itbiznes
https://t.co/YLvQK1148E: 1
@ciachopl @PaczkomatyPL Daj szansę chłopakom. Zakręceni przed Świętami. 😉
Tymczasem zdrówka!: 2
Kolejna aktualizacja :)
Wczoraj zadzownil @PaczkomatyPL - odnaleźli moją przesyłkę, i proszą o adres aby kurier przywiózł bo „nie mogą paczki włożyć do paczkomaty”.
Podalem adres i…dzisiaj paczka została wsadzona do paczkomatu..

Jestem na izolacji, więc w sobotę sprawdzę 
10/x: 0
Tylko w ubiegłym roku w Polsce wysłanych zostało ponad 640 mln paczek, w tym będzie ich jeszcze więcej. Niestety opakowania są najsłabszym punktem branży #ecommerce. #InPost wraz z #CCC i #Modivo zmienia zasady gry! @PaczkomatyPL @RBrzoska @BriefPolska 
https://t.co/RnY4wb5y1r: -1
@Marek2137 @KulagaWojciech @pieinggg @PaczkomatyPL Jeżeli jest to cześć wyłączona z użytku to nikomu nie utrudnia przejścia bo nikt nie ma prawa tam przechodzić: -2
@luizaroslon @pieinggg @PaczkomatyPL Dzięki za informację: 0
"Walniemy na nim pro-eko-mural i będzie w pyte" ~@PaczkomatyPL https://t.co/1Y7N5TcV2j: 0
@KulagaWojciech @pieinggg @PaczkomatyPL Paczkomat jest przy Gagarina 8 w Warszawie. Przejście do tej pory było całkowicie dostępne. Obecnie paczkomat jest już podłączony, oświetlony i ma aktywny ekran.: 1
@KulagaWojciech @pieinggg @PaczkomatyPL Nawet jeśli jest tymczasowa to wciąż utrudnia przejście np niepełnosprawnym: 0
@pieinggg @PaczkomatyPL Można prosić adres Paczkomatu? Bo ta taśma sugeruje, że nie jest to działająca maszyna, a przejście z jakiegoś względu wyłączone z użytku także fajnie jakbyśmy mogli zweryfikować, że faktycznie to nie jakaś uzgodniona i tymczasowa sytuacja.: 2
Ktoś na wózku, rodzice z dziećmi, ktoś o kulach, z walizką - muszą być ZACHWYCENI.

No @PaczkomatyPL  tego się po Was nie spodziewałam. https://t.co/xrXt6mJwds: 2
@SquashPL @PaczkomatyPL zapewniamy, że należność zwrócimy, tak jak obiecaliśmy. Przepraszamy jeszcze raz za tę sytuację.: -1
@zakino4 @PocztaPolska @PaczkomatyPL Tak! @PocztaPolska wiezie moj list polecony priorytet od 22.11 do Krzeszowic cały czas! A co do @PaczkomatyPL to lepiej przekierowywać do tych szaf co ich tyle poustawiali niż spotkać się z kurierem.: 2
@Allegro_Group @PaczkomatyPL Gdyby ktoś był ciekaw jak sytuacja się potoczyła. InPost nie mógł pomóc, „proszę o kontakt z Allegro”. Allegro poprosiło o zakup etykiety na własną rękę i mają odliczyć od rachunku. Czekam +- tydzień i wciąż należność nie została odliczona 👌

Może do świąt się uda!: 2
@InPostPL1 trochę powaliło... Wstawili @PaczkomatyPL w jednej z warszawskich bram...

#shame 🤦🤦‍♀️🤦‍♂️ https://t.co/4lWPwR6jjU: 0
@PaczkomatyPL Mogę prosić jakiś link ?: 0
Obsługa paczek, listów przez @PocztaPolska to jest jakiś inny świat. Tym bardziej nie ogarniam, który geniusz wpadł na pomysł, żeby paczki @PaczkomatyPL przekazywać do odbioru w punktach Poczty Polskiej. 

To jest jakiś inny poziom abstrakcji. Nie mam już sił na paczkomaty… https://t.co/eiNISFKIl0: 1
RT @OLogistyczny: @PaczkomatyPL i #MODIVO wprowadzają opakowania zwrotne wielokrotnego użytku. Dzięki temu spółka zminimalizuje zapotrzebow…: 0
@PaczkomatyPL i #MODIVO wprowadzają opakowania zwrotne wielokrotnego użytku. Dzięki temu spółka zminimalizuje zapotrzebowania na jednorazowe opakowania nawet dziesięciokrotnie. 

#ecommerce #logistyka 

https://t.co/XtwSfHKDGc: 0
@PaczkomatyPL dlaczego kłamiecie w #Fakty, że komunikujecie się tylko za pomocą aplikacji? Ciągle otrzymuje maile i smsy. Zweryfikujcie swoje działania.: 1
@PaczkomatyPL z kolejną świetną inicjatywą - ekologiczne podejście ma znaczenie :)

https://t.co/cOCmH76emc: 1
@PaczkomatyPL jak zgłosić awarię paczkomatu? Od dłuższego czasu mój ulubiony paczkomat ma problem z dotykowym ekranem :(: -1
@PaczkomatyPL A kiedy to nastąpi?: 0
@PaczkomatyPL Trzy tweety wyżej.: 0
@PaczkomatyPL Hej, został 10 dni do Bożego Narodzenia. Zdążycie?: 0
Bardzo dobry pomysł InPost @PaczkomatyPL  a wy morele @morele_net  i x-kom wstydźcie się @xkom_pl  może pora na was ?! I coś z tym zrobicie?! Czas na Ekologię!! https://t.co/rstNGGdbJt: 1
@JanSpiewak @restory244 @PaczkomatyPL to fake? Mam nadzieje… jak przejechac wozkiem?: -3
@PaczkomatyPL @InPostPL1 a co to się tutaj dzieje?

https://t.co/wGdBUolKtL: 0
@PaczkomatyPL Zgadzam się.: 0
@PaczkomatyPL muszę przyznać, że bardzo to słabo u Was działa... trzy umowy w ostatnim czasie, 2 różnych kurierów, za każdym razem ta sama gadka. Może @mBankpl powinien pomyśleć nad alternatywą?: -1
@ags313 @kuba_suder @PaczkomatyPL jeszcze do niedawna w paypalu był jakiś dziwaczny limit 18 znaków na hasło. Także jak światowy serwis finansowy ma takie głupie limity, to mniej dziwi inpost.: -1
@karolina_anna1 @donaldtusk @anneapplebaum @empik @PaczkomatyPL We're looking forward to hear from you about this book Karolina.: -2
RT @karolina_anna1: Kolejna paczka książek pod choinkę.
#Wybór @donaldtusk @anneapplebaum 
via @empik @PaczkomatyPL ;) https://t.co/ZeinZGL…: 0
Dlaczego @PaczkomatyPL nie wprowadził nadawania paczki w paczkomacie za pomocą otwierania skrytki telefonem? Paczki można odbierać zdalnie, ale nadawać już nie można 🤦 Dodatkowo pandemia trwa nadal..: 0
Kolejna paczka książek pod choinkę.
#Wybór @donaldtusk @anneapplebaum 
via @empik @PaczkomatyPL ;) https://t.co/ZeinZGL51N: 0
@KaTaRkReaktywa1 @PaczkomatyPL Ja regularnie odbieram 20kg karmy dla psa z najwyższego piętra 😀 zerkając z nadzieją przy otwieraniu na te pozwalające się czołgać 😀: 1
RT @wiadspozywcze: #InPost razem z modową platformą e-commerce spółki @eobuwie_pl z @GrupaCCC  rozpoczyna pierwszy na polskim rynku pilotaż…: 1
RT @eNewsroom: Pierwsze na rynku opakowanie zwrotne dedykowane dla #ecommerce! Dostarczy je #InPost, który razem z @cccshoesbags i #Modivo…: 0
@PaczkomatyPL ponownie wykorzysta zużyte kartony swoich klientów! 👏🏼  Zmniejszając tym samym ślad węglowy paczek, nawet 10 cykli per opakowanie 💪🏼♻️

https://t.co/uCoU8Ssqin: -2
@ags313 @kuba_suder @PaczkomatyPL Tak jest też w @SkyCashPoland w aplikacji mobilnej wpisujesz krótsze hasło a na stronie pełne.: 0
./out/82349ce6-7fc2-4de7-9595-895fa6a1980e.zip is empty! No sentiment for value 586
@QbaMkn86 @PaczkomatyPL Peszek: no sentiment
@KaTaRkReaktywa1 @PaczkomatyPL Super!!!🤣: 1
RT @PaczkomatyPL: Synoptycy zapowiadają w tym tygodniu obfite opady Paczkomatów ❄️
Pojawiły się już w nastepujących miejscowościach:
📦 Chor…: 0
@PaczkomatyPL @TesTeq A rabacik?: 0
@TesTeq @PaczkomatyPL @RBrzoska @infel1 @szaffi 11 lat temu, kiedy Pan @RBrzoska rozpędzał swoją rewolucję, napisałem do kolegi z podobnej firmy pakującej reklamowane towary, że jeśli uda mu się wymyślić coś na kształt palety i kontenera, ale z obsługą telefonem, to zawojuje świat. I chyba jest na ścieżce... 👍😃: 1
@SpidersWebPL Niech @PaczkomatyPL lepiej najpierw dostarczą mi moją przesyłkę, obawiam się że jeśli z tym już mają problemy w okresie świątecznym to wymienne pudełko może ich zdecydowanie przerosnąć 😏: 0
./out/60b10d62-818b-439d-8a40-78ee0eb3729a.zip is empty! No sentiment for value 592
@KaTaRkReaktywa1 @PaczkomatyPL https://t.co/8OIkiVeMKM: no sentiment
📢📢📢

#NieDajSięZłowić 

@CERT_Polska @InPostPL1 @PaczkomatyPL #CyberSecurity #phishing https://t.co/uRfBBMMNd4: 0
@KozubG @PaczkomatyPL A, w bankach to akurat normalka… ale tam jest trochę inaczej, bo jeszcze często się wpisuje pojedyncze wybrane litery, więc oni to pewnie faktycznie zapisują w całości niehaszowane: -2
@TesTeq @PaczkomatyPL @RBrzoska @infel1 @szaffi Pomysł jest rewelacyjny. Aura covidowa zapewne mu trochę przeszkodzi, ale powinien mieć wszędzie zielone światło. Dostałem dziś książkę - w superoszczędnym  opakowaniu kartonowym, które by mogło jeszcze służyć 20 razy. Ale nie ma komu oddać, nawet za friko.: 1
@kuba_suder @PaczkomatyPL Pół biedy z InPost. Ostatnio takie samo ograniczenie spotkałem w PKO 🙈: -4
RT @TesTeq: 👍 Jestem pod wrażeniem! Opakowania zwrotne @PaczkomatyPL! Brawo @RBrzoska! @Orginal_Replica @infel1 @szaffi https://t.co/TqWEfS…: 0
@TesTeq @PaczkomatyPL @RBrzoska @Orginal_Replica @infel1 @szaffi Genialne!: 1
@kuba_suder @PaczkomatyPL Wiesz jakie bylo moje zdziwienie, jak gdzies wpisywalem 20 znakow, a potem nie moglem sie nimi zalogowac? Odrobine mniejsze, niz jak sie dowiedzialem, ze jest limit 16. I znacznie mniejsze niz jak sie zalogowalem pierwszymi 16oma z 20 liter.: 0
@infel1 @PaczkomatyPL @RBrzoska @Orginal_Replica @szaffi Ja wysyłam książki sprzedawane na Allegro w kartonach po książkach, które do mnie przyszły.: 0
@PaczkomatyPL Uprzejmie dziękuję za powiadomienie, że przesyłka dotarła dziś do paczkomatu. Czy istnieje jakiś powód, dla którego otrzymuję to powiadomienie co 10 minut?: 0
@TesTeq @PaczkomatyPL @RBrzoska @Orginal_Replica @szaffi Problem z kartonowymi paczkami oczywiście mam.
Nie bardzo wiem co z nimi robić, ale żeby używać "po kimś" to fe ;): -1
@infel1 @PaczkomatyPL @RBrzoska @Orginal_Replica @szaffi Nie da się ukryć, że będą tacy, którzy będą woleli spalić pudełko w piecu niż zanieść je do paczkomatu. 🤪: -1
#MODIVO jako pierwsze dołącza do innowacyjnego projektu #InPost i wprowadza bardziej ekologiczne rozwiązanie logistyczne!
Opakowanie każdej przesyłki, którą zamówisz w MODIVO, możesz zwrócić do najbliższego @PaczkomatyPL i dać mu kolejne życie. @eobuwie_pl
https://t.co/RH3GxD3e1P: 2
@kuba_suder @PaczkomatyPL Powyzej 16 znakow jest zbyt bezpiecznie.: 0
@PaczkomatyPL Zarówno ja jak i TEORETYCZNIE osoba z numerem telefonu który został podany ;): 0
Synoptycy zapowiadają w tym tygodniu obfite opady Paczkomatów ❄️
Pojawiły się już w nastepujących miejscowościach:
📦 Chorkówka,
📦 Maszkienice,
📦 Hajduki Nyskie,
📦 Glincz,
📦 Piętno 🔥
A jak Wasza pogoda na odbiór? 😄 https://t.co/yEcyw48yUB: -1
@Krzysiec @PaczkomatyPL Chyba tak, ja ostatnio tak miałem, że ktoś podał mój numer telefonu... Na infolinii dowiedziałem się, że jeśli ma e-mail to odbierze paczkę bez problemu, a smsa zignorować.: 0
@Krzysiec @PaczkomatyPL Ja raz tak miałem i na mailu będzie miał kod i odbierze: -1
RT @TesTeq: 👍 Jestem pod wrażeniem! Opakowania zwrotne @PaczkomatyPL! Brawo @RBrzoska! @Orginal_Replica @infel1 @szaffi https://t.co/TqWEfS…: 0
Hej @PaczkomatyPL ! Czy jeśli nadawca podał dobrego maila i paczkomat ale walnął się w numerze telefonu to będę w stanie odebrać paczkę z paczkomatu?: 0
@TesTeq @PaczkomatyPL @RBrzoska @Orginal_Replica @szaffi Inpost jest super, ale te papierowe opakowania, wielokrotnego użytku, to czysty marketing, na który jestem odporny.🤷‍♂️: 2
Pierwsze na rynku opakowanie zwrotne dedykowane dla #ecommerce! Dostarczy je #InPost, który razem z @cccshoesbags i #Modivo rozpoczyna pilotażowy test, w którym da wspólnym klientom dostęp do ekologicznych opakowań. @RBrzoska @PaczkomatyPL #paczkomaty 
https://t.co/w6zRXgfbdw: 0
👍 Jestem pod wrażeniem! Opakowania zwrotne @PaczkomatyPL! Brawo @RBrzoska! @Orginal_Replica @infel1 @szaffi https://t.co/TqWEfSBkpd: 0
@PaczkomatyPL @kuba_suder Ograniczenie długości hasła żeby dało je się zapamiętać to nielogiczny pomysł: przecież to użytkownik wybiera hasło takie jakie woli. Poza tym, że wielu jak @kuba_suder i ja, używa password managerów. Zapamiętywalne hasła są mniej bezpieczne.: -4
@PaczkomatyPL Wrzuciłem link, samo kliknięcie w niego nie wydaje się szkodliwe, chyba.: -1
@PaczkomatyPL Taki limit nasuwa od razu podejrzenie, że hasło jest zapisywane bez zmian, czego absolutnie nie wolno robić. Normalnie z hasła oblicza się tylko tzw. hash, który ma stałą długość, czy hasło wejściowe ma 10 czy 60 znaków, więc długość nie powinna mieć znaczenia (oprócz minimalnej): -1
@PaczkomatyPL Fajnie, ale zgłoście im, żeby to jednak przemyśleli, bo nie robi się tak w dzisiejszych czasach… 99% stron przyjmuje takie hasła, i jeśli jest taki limit długości to generalnie jest to 🚩🚩🚩. Dłuższe hasło jest bezpieczniejsze i nie zawsze trudniejsze do zapamiętania/zapisania.: 1
@KaTaRkReaktywa1 @PaczkomatyPL Ja od czasu paczki na wysokościach, po odbiór wysyłam męża do paczkomatu.: 0
@KaTaRkReaktywa1 @PaczkomatyPL ciesz się, że nie kazali Ci urosnąć 30 cm jak mi, gdy miałam 20 kg wyciągnąć: 1
@PaczkomatyPL Uczulajcie kurierów ze najwyższe i najniższe miejsca to tak słabo jak są dopchniete do końca: -2
.@PaczkomatyPL jest jakiś dobry powód, dla którego nie przyjmujecie hasła dłuższego niż 16 znaków (w 1Password generują mi się standardowo 20-znakowe)? i mam nadzieję, że nie jest to powód "bo zapisujemy te hasła czystym tekstem w bazie" 😅 (BTW, animacja z 👀 jest genialna) https://t.co/Oe3XNR2qvI: 2
@BorowieckaEwa @PaczkomatyPL Moja mama tak miala ze przechodniow prosiła o pomoc bo było tak samo tylko na górnej: 1
@BorowieckaEwa @KaTaRkReaktywa1 @PaczkomatyPL Lepiej jak niziołek kucać
Niż drabiny w polu szukać :D: 0
@KaTaRkReaktywa1 @PaczkomatyPL Lepiej kucać niż szukać drabiny jak dla mnie. 😂: 1
@komputerow @CERT_Polska @CERT_OPL @PaczkomatyPL Zbieramy też jako @CSIRT_KNF , dzięki 👍: 1
Firmy e-commerce wypowiadają wojnę nadmiarowi opakowań
@PaczkomatyPL 
https://t.co/T8dVBx695a: -1
@KaTaRkReaktywa1 @PaczkomatyPL Pech: -1
@promycz3k kurde to nie wiem :(( @PaczkomatyPL pomóżcie koleżance: 0
@KaTaRkReaktywa1 @PaczkomatyPL "Strefa komfortowego odbioru" czy jakoś tak? :|: 1
Hej jestem Wam wdzięczny @PaczkomatyPL ze dbacie o moje wytrenowanie i umiejetność czołgania 

#Paczkomaty https://t.co/UcL3sVbCqz: 1
@PaczkomatyPL @PaczkomatyPL jeden z Waszych kurierów znów parkuje nielegalnie w tym samym miejscu. Skrzyżowanie Kolberga i Krowoderskiej w Krakowie, 13.12.2021 godz. 12:23. Rozumiem, że zainterweniowaliście w przypadku poprzedniego kierowcy, ale to chyba bardziej powszechny problem. https://t.co/s0ZxmZ9EBN: 0
@RBrzoska Panie Rafale, a co z folią opakowaniową (LDPE). Jestem jej recyklerem i może coś razem wymyślimy? (Odbierający paczki umieszczają folię w paczkomacie, kurier zabiera ją do centrum dystrybucyjnego, belujemy, odbieramy, robimy regranulat i folię/ worki). @RBrzoska @PaczkomatyPL: 0
RT @Greennewspl1: Nowa zielona inicjatywa InPost @PaczkomatyPL. Spółka Rafała Brzoski oferuje wtórne wykorzystanie opakowań.
#LessWaste

ht…: 0
Nowa zielona inicjatywa InPost @PaczkomatyPL. Spółka Rafała Brzoski oferuje wtórne wykorzystanie opakowań.
#LessWaste

https://t.co/27OzMlOCGU: 0
Ekologiczne opakowania wielokrotnego użytku by @PaczkomatyPL ! projekt powstał przy współpracy z #Modivo.

https://t.co/fu3S7uhATb: 0
#InPost razem z modową platformą e-commerce spółki @eobuwie_pl z @GrupaCCC  rozpoczyna pierwszy na polskim rynku pilotażowy test, w którym przesyłki będą dostarczane klientom w prawdziwie ekologicznych opakowaniach. Na czym to polega? @PaczkomatyPL 

https://t.co/oYH5Zsp1yH: 0
Czy cyfryzacja może pomóc przywrócić pozrywane łańcuchy dostaw? W ramach cyklu #CyfrowaTransformacja z partnerami: @otomotopl i @PaczkomatyPL zastanawiamy się jak zarządzać logistyką niespokojnych czasach.
Więcej dziś w „Pulsu Biznesu” oraz na https://t.co/cwu7iyvEHe

Zapraszamy! https://t.co/Vte2Dn5tfQ: 0
@PaczkomatyPL Planujecie zrobić opcje zgłaszania uszkodzonych paczkomatow z poziomu aplikacji?: -2
#Bezpieczeństwo https://t.co/VICxoWnfBU: 1
@komputerow @CERT_Polska @CERT_OPL @PaczkomatyPL My też dziękujemy. I już reagujemy! :): 0
@ChybaMaks @sizeer @PaczkomatyPL uważam że jedynym słusznym rozwiązaniem w tej sytuacji będzie ratio. https://t.co/GvxxTRS64J: 2
@TWINZ1FN @ChybaMaks @sizeer @PaczkomatyPL chodziło mi o to ze gorzej jak by były jakies inne buty na resell. ofc af1 z sizera nie sa na resell: -1
@ChybaMaks @sizeer @PaczkomatyPL Dobra już te wszystkie aspekty że porwane pudełko itp to mam jedno pytanie jakie af1 kupiłeś XD?: 1
@PaczkomatyPL @genesispr_ Pomóżmy tej starszej kobiecie, która jest w dramatycznej sytuacji zebrać pieniądze na remont dachu, bo w przeciwnym razie w zimie zamarznie ona z zimna. Dobro wraca. Warto pomagać. Szczególnie teraz w okresie adwentu. https://t.co/h3RvtUudUN 
UDOSTĘPNIJ!!!: -2
@Chudyini @ChybaMaks @sizeer @PaczkomatyPL Na resell z sizera to jakas okazja: 1
Szanowny panie @PaczkomatyPL, nie musisz dzwonić, ja tę paczkę już dawno odebrałem. https://t.co/xqfccp2iZ1: 1
Hej @PaczkomatyPL , ile można naprawiać paczkomat?🤔
Może wypadałoby chociaż wyjąć paczki i przerzucić do innej maszyny, a nie tak trzymać od wtorku, 6ty dzień..?: 0
@ChybaMaks @sizeer @PaczkomatyPL jak z butami nic sie nie stało to chyba git gorzej jak by to były buty na resell: -1
@ChybaMaks @sizeer @PaczkomatyPL zabrać chcieli pewnie: 0
./out/e9d04e5d-c6d3-4146-9972-0476ac57a5c5.zip is empty! No sentiment for value 651
@PaczkomatyPL 🤷‍♂️: no sentiment
@ChybaMaks @ZaniuFN @sizeer @PaczkomatyPL No co kurier chciał se przymierzyć i cyknąć foto takie dripowe: 0
@ChybaMaks @sizeer @PaczkomatyPL JA BYM TEGO NIE PRZYJĄŁ: 0
Uwielbiam zaglądać w szczegóły przesyłki z AliExpress zamówionej do @PaczkomatyPL i widzieć, że jest ona realizowana przez @PocztaPolska 🤦‍♂️ W minionym tygodniu jedna taka do mnie trafiła tylko dzięki uprzejmości kuriera, teraz widzę, że kolejna jest w drodze.: 1
@Artur09739581 @reus00971vol2 @PaczkomatyPL Cały czas tak robią i nawet w zeszłym tygodniu tak miałem, więc nie wiem co za problem był żeby powtórzyć ten manewr.: 2
@reus00971vol2 @B_Maniek @PaczkomatyPL Wynika to z tego, że zajęte są wszystkie skrzynki w paczkomacie. Zwolnią się to przy następnym podjeździe kurier i załaduje, kiedyś w takich przypadkach zdarzało się , że umieszczali paczkę w sąsiednim paczkomacie.: 1
@B_Maniek @PaczkomatyPL O, u mnie 8:17 i podobny problem :) Najlepsze, że były dwie paczki wydane do doręczenia o tej samej godzinie, jedna po 10 wylądowała w paczkomacie, a druga - ktokolwiek słyszał, ktokolwiek wie: 1
Hej @PaczkomatyPL, czy jest szansa, że jeszcze dzisiaj dostarczycie paczkę wydaną do doręczenia dziś o 7:17 rano?: 1
@wodoodporny @kordaal @PaczkomatyPL O to ja nie miałam takiej sytuacji przy paczkach z Allegro stąd mój wniosek ;): 0
@PaczkomatyPL Grze, jak i książce nic się nie stało. Odkleiłam taśmę firmową od spodu paczki, ledwo podniosłam nad podłogę, dziesięć centymetrów dosłownie, i dół zaczął się otwierać. Gdybym podniosła wyżej, to by mi wszystko z niej wypadło: -3
@PaczkomatyPL Nie zwalam winy na karton, ani tego kto to pakował, bo zapakowanie planszówki do takiego kartonu, to sztuka. Pod grą było takie powietrze paczkowane? I nad tym nie ubolewam, lecz nad stanem przesyłki już tak: 0
@PaczkomatyPL Nie, na szczęście nie, ale samo zapakowanie paczki, jak i przyklejenie taśmy od spodu kartonu, to jakiś żart za przeproszeniem. Gra, którą zamówiłam, była wsadzona, że praktycznie opierała się na kartonie, była ledwo wsadzona, a książka owinięta w folie bombelkową, nie tak źle: -2
@maciej_zych23 @PaczkomatyPL @InPostPL1 🙃🙃🙃: 0
@LukeCue @PaczkomatyPL @InPostPL1 Masz paczkomat na ulicy Pokątnej? Szanuję :) https://t.co/WUd2f4yq8d: -1
@maciej_zych23 @PaczkomatyPL @InPostPL1 I odebrane 👌🏻 https://t.co/gR4CF3Y6Vh: 0
@_JaSemJJ_ @PaczkomatyPL ZDECYDOWANIE!: 0
@martilovesshawn @PaczkomatyPL Obawiam się że mają konkurs "kto kopnee mocniej dostaje premie": 2
Błagam, kurierzy @PaczkomatyPL powinni płacić należne za chyba dosłownie skopanie mi paczki, bo dopiero jak przyjechałam do domu, to zwróciłam uwagę. Była włożona "dziurą" do tyłu, więc nie widziałam. Widziałam tylko ładniejszą stronę, plus, chwila moment i paczka by była otwarta https://t.co/EOxXKTLjxS: 0
@wpisztutajnic @PaczkomatyPL Ja dostałem kiedyś paczkę w niedzielę :): 0
@Tomasz_Zaton @michuk @tmrohan @UiPath @Wise @PaczkomatyPL For sure :): 8
@arsenisan @michuk @tmrohan @UiPath @Wise @PaczkomatyPL Networking always pays off: 0
@maciej_zych23 @PaczkomatyPL @InPostPL1 Dzisiaj: 0
@LeszekJarema @maciej_zych23 @PaczkomatyPL Dobrze by było, bo podoba mi się ☺️: 0
@misia_domi @kordaal @PaczkomatyPL Nie wszystko z Allegro też idzie w sobotę bo czasami system nie załapie bo u mnie dwa razy na weekend nie poszła pomimo że warunki na dostawę w weekend spełniała ¯\_(ツ)_/¯: -1
@kordaal @PaczkomatyPL Zależy od kogo dostaje paczkę 😅 nie wszystkie są wrzucane w sobotę. Te z Allegro na pewno są, a reszta to zależy 😅: 1
@LukeCue @maciej_zych23 @PaczkomatyPL Pewnie ForzaH5: 0
@maciej_zych23 @PaczkomatyPL @InPostPL1 Tak powinieneś dostać. Moja właśnie dziś trafiła do skrytki. Amazon wysłał mi pakę na weekend a nawet się o nią nie prosiłem. https://t.co/FbGA2QGg1V: -1
@maciej_zych23 @PaczkomatyPL @InPostPL1 Dziś dostałem to samo z rana i dziś odbieram na 99%.: 0
@LeszekJarema @maciej_zych23 @PaczkomatyPL A mój ma przyjść pod choinkę. Ciekawe tylko jaki, bo Mikołajkowa nie chce się wygadać.: 1
@michuk @tmrohan @UiPath @Wise @PaczkomatyPL My best guess it is due to norms that local VCs are dictating throughout marketing channels. This creates a feeling that “there is no other way around.” So founders who do not research alike startups, nor hangout with VCs from UK/US/DACH have no idea about possible valuations.: 2
@maciej_zych23 @PaczkomatyPL 🤫 też mam https://t.co/5DHFl91BSn: 0
No i jest. Piękności. Dzięki @PaczkomatyPL 😎
#Xbox20 https://t.co/C4H50OOciQ: 0
@tmrohan They are still in the $1-5M range in Central and Eastern Europe, the region that produced the biggest VC returns in the last decade globally with huge wins like @UiPath @Wise or @PaczkomatyPL. 

Funny how not competitive the region still is compared--lots of space to grow!: 2
@pawelsz5 @kordaal @PaczkomatyPL Ja nawet kilka razy w niedzielę miałam.: -1
@kordaal @PaczkomatyPL Od bardzo długiego czasu paczki dostarczają w soboty. Wszystko zależy od przepełnienia paczkomatu.: 0
@pieinggg @PaczkomatyPL W sensie fotowoltaika? Ten nie ma. Zresztą i tak nie miałby sensu bo stoi w cieniu pod ścianą.: 0
@wpisztutajnic @PaczkomatyPL Mi czasami przychodziły nawet w niedzielę!: 0
Może @PaczkomatyPL mają energię elektryczną wytworzoną CZARAMI.👀 https://t.co/FZZes4hSjy: 0
#InPost il peggior servizio di spedizione in assoluto. PESSIMO, da evitare,
@PaczkomatyPL: 0
@lapinski_pawel @PaczkomatyPL a one nie mają paneli OZE?: 0
@PaczkomatyPL A to dziwne, bo ostatnio jedna paczka czekała mi do poniedziałku na oddziale :): 0
@maciej_zych23 @PaczkomatyPL @InPostPL1 Dziś ;) ja dostałem w ostatnią niedziele nawet 👌: 0
‼️UWAGA
Kolejny numer na który trzeba uważać: 514 014 922 
Próba wyłudzano poprzez płatności @PaczkomatyPL 
Numer cały czas aktywny @PolskaPolicja https://t.co/xvUX604yiU: 1
@maciej_zych23 @PaczkomatyPL @InPostPL1 U mnie też dwie w doręczeniu i się zdziwiłem. Może w okresie przedświątecznym tak robią.: 0
@maciej_zych23 @PaczkomatyPL @InPostPL1 Dzisiaj: 0
@wpisztutajnic @PaczkomatyPL Ja zamawiałem z allegro z przewidywana dostawą na piątek-poniedzialek za tydzień, rano się budzę a o 8:30 paczka w paczkomacie. Dwie kolejne paczki z dostawą na poniedziałek już na mnie czekają w multiskrytce ;): 0
@maciej_zych23 @PaczkomatyPL @InPostPL1 Ostatnio wszystko bez względu na rodzaj dostarczają w weekend nawet ;): 1
@maciej_zych23 @PaczkomatyPL @InPostPL1 Będzie dzisiaj: 0
Jak to jest z @PaczkomatyPL ? Jeśli mam informację „paczka została wydana do doręczenia” otrzymaną dzisiaj to rzeczywiście doręczenie będzie dzisiaj czy jednak dopiero w poniedziałek? :) 
#inpost 
@InPostPL1 @PaczkomatyPL https://t.co/EziQ7nKex5: 0
@Emill1984 @SledziuStream @wpisztutajnic @PaczkomatyPL Na fejsbuku o Panam wypisuje menda!: 0
@wulgarny_gracz @SledziuStream @wpisztutajnic @PaczkomatyPL a Tobie co sie tak po roku przypomnialo? xD: 0
@SledziuStream @wpisztutajnic @PaczkomatyPL Paczaizm + Kombinat: 0
@wpisztutajnic @PaczkomatyPL Bywa różnie ale często zdarzało mi się odbierać w sobotę rano.: 0
@PaczkomatyPL No śmiało. Niech pochwalą się też ci, którym działa zdalne otwieranie skrytek...: 0
@PaczkomatyPL Ooo to mniej prezentów wysyłają;)? Ale 6 czy 7 dni w tygodniu? ;): 0
@kr4wi3c @PaczkomatyPL Moja też już dostarczona :D: 0
@wpisztutajnic @PaczkomatyPL Możliwe. Na to wychodzi że inpost dzisiaj normalnie jeździ. 
My dostaliśmy przed chwilą powiadomienie że paczka została umieszczona w paczkomacie. 
Przed chwilą też dostałem info od kumpla że ma do odebrania przesyłkę z paczkomatu.: 2
@wpisztutajnic @PaczkomatyPL Paczkomaty obecnie działają 6 jak nie 7 dni w tyg. Masz do odbioru tylko 24 godziny. Święta.: 0
@PaczkomatyPL o proszę, akurat dostałem powiadomienie że dotarła &lt;3 dobra robota, dostarczona w 15 godzin &lt;3: 0
./out/019d590d-0b4a-4e4e-8e60-5db7127e8a34.zip is empty! No sentiment for value 710
@PaczkomatyPL https://t.co/Zt9VV2x3Pl: no sentiment
@wulgarny_gracz @wpisztutajnic @PaczkomatyPL XDDDDDD: 0
@SledziuStream @wpisztutajnic @PaczkomatyPL Dobrze że Cię spotkałem - odpierdol się od Panam!: 0
@wpisztutajnic @PaczkomatyPL zwykle jak "wydano do doręczenia" to kurier ma tego dnia na pace... czy do paczkomatu wsadzi to zależy czy w paczkomacie ma miejsce ;): 0
@wpisztutajnic @PaczkomatyPL Paczkinat brzmi jak jakiś artefakt...: 0
@wpisztutajnic @PaczkomatyPL No to teraz tylko trzymać kciuki za weekendową radość...: 1
@Evreal @PaczkomatyPL nie, nigdy :D ale dawno nic nie zamawiałem z allegro.: 0
@Thot697 @PaczkomatyPL ta, przez Smarta zamawiałem: 0
@wpisztutajnic @PaczkomatyPL Nie dostałeś nigdy paczki w weekend? 
Zdarzyło mi się odbierać paczkę po jej dostarczeniu już w sobotę wieczorem, a nawet niedzielę przed południem.: 0
@wpisztutajnic @PaczkomatyPL Mnie ostatnio przychodzily do paczkomatu w soboty, więc pewnie tak.: 0
@wpisztutajnic @PaczkomatyPL Jeśli masz allegro smart to dojdzie: 0
@wpisztutajnic @PaczkomatyPL Tak, normalnie w sobotę pakują do paczkinatow: -1
@wpisztutajnic @PaczkomatyPL Z mojego doświadczenia to rulekta.: 0
Jeśli w apce paczkomatów mam info, że paczka została wydana do doręczenia dziś rano, to trafi dzisiaj do paczkomatu mimo tego że jest sobota? @PaczkomatyPL: 0
@lapinski_pawel @PaczkomatyPL Osoba, która wymyśliła paczkomaty powinna dostać Pokojowego Nobla.: 0
@RBrzoska @wboczon @PaczkomatyPL Panie Rafale...mi pana pracownicy chcieli połamać telefon przy paczkomacie więcej info na biuro@inpost.pl.
Może ze swojej pozycji też Pan zadziała w tej sprawie.... Pozdrawiam serdecznie: 1
Szanowni @PaczkomatyPL czy paczkomaty mają akumulator? Bo na całym osiedlu wysiadł prąd a paczkomat działa https://t.co/BvN2zPfKAy: 1
@PiotrO1906 @WislaKrakowSA @PaczkomatyPL Ogólnie wysyłka paczkomatem czy kurierem jest x2… zapewne klub sobie dodatkowo zarabia na wysyłce, trochę nie fair wobec kibiców.: 2
@PaczkomatyPL Poszło na DM: 0
@WislaKrakowSA Piękna rzecz, ale żeby w obecnych czas nie dogadać się z @PaczkomatyPL odnośnie tańszej wysyłki to trochę słabe :D: 0
@PaczkomatyPL Ok. Zaraz podeśle szczegóły: 0
Hej @PaczkomatyPL w jaki sposób można złożyć skargę na waszego kuriera? Protokół reklamacyjny na waszej stronie czy w inny sposób?: 0
@PaczkomatyPL @genesispr_ Brawo 😊 https://t.co/jiqyCSEQm3: 0
@PaczkomatyPL @genesispr_ gratulacje!💪🏻: 0
@PaczkomatyPL Dziękuję za chęć pomocy, tylko niestety wydaje mi się ze po prostu to błąd kuriera. Wyjął nie tą paczkę....  Oby więcej takie coś mnie nie spotkało. Przykro bo to przed świętami.: 2
@PaczkomatyPL Rozmawiałam z nadawcą,mówi że taka sytuacja nie miała miejsca.Paczka przejechała całą Polskę,została umieszczona w paczkomacie i dopiero po 2h zorientowaliście się ze coś nie tak?A jakbym o g.11 ją odebrała?Co wtedy z Waszymi procedurami?Nie fajnie postępujecie @PaczkomatyPL: 1
./out/1818dd8b-4dff-48d5-8ed7-eea2adcf0688.zip is empty! No sentiment for value 736
@PaczkomatyPL 656200756145864115810185
😔: no sentiment
@PaczkomatyPL  "dziękuję" za 12 godzi na odbiór paczki. Tak informacja powinna przyjść dodatkowym SMS + 1000 wykrzykników. Paczka trafiła do paczkomatu 00:27 -&gt;nieświadomy zmian nie odebrałem jej z rana -&gt; w pracy dostaję info, że wraca do nadawcy 😡: 1
@PaczkomatyPL znowu zawiedliście. Dostałam powiadomienie że mogę odebrać przesyłkę z paczkomatu o g.11. O g. 13 ponowne powiadomienie że przesyłka została zwrócona do nadawcy. Nadawca nic o tym nie wie. Infolinia @PaczkomatyPL każe się skontaktować z nadawcą. To jakaś kpina!!!: 0
Wtf dlaczego bez jakiegokolwiek uprzedzenia zmieniają mi paczkomat i teraz musze zapieprzać do jakiegoś 10 km dalej?! 
Inpost @PaczkomatyPL robicie to ŹLE: -1
@Garm1981 @PaczkomatyPL Przecież żadna Twoja wina! Mam nadzieję, że się jeszcze znajdzie pod siedzeniem kuriera, a jak nie to widać ten Zajdel nie jest mi pisany 😉: -1
@Inferno889 @PaczkomatyPL Masz rację. Nie wierzę...

Bardzo przepraszam za niedogodności, nigdy nie miałem do czynienia z taką sytuacją :-(: 0
@MastercardPL 
#IdeaGetinLeasing
@wwwitakapl
@Provident_PL
@SuperbetPL
@TDJ_SA
#GetinNobleBank
@PwCPolska
#LPP
@IrenaEris
@ING__Polska
@stspolska
@DunBradstreetPL
#FIGENEcapital
#FiveRand 
@Test_Army
@PaczkomatyPL 
#PolskaLigaSiatkówki
@BNPParibas_PL 
#KAPholiding
...
2/3: 0
@Garm1981 nie uwierzysz, @PaczkomatyPL zgubił książkę od Ciebie...8 grudnia rano wydano książkę kurierowi do doręczenia i kamień w wodę.....Reklamacja złożona :(: 0
@PaczkomatyPL co to ma być xdd https://t.co/IZ9vBuJ5T1: 0
@PaczkomatyPL @genesispr_ Gratulacje: 0
RT @PaczkomatyPL: #ZłoteSpinacze za nami! W tegorocznej edycji Gali, wraz z @genesispr_ , zostaliśmy nagrodzeni w kategorii "PR finansowy i…: 1
#ZłoteSpinacze za nami! W tegorocznej edycji Gali, wraz z @genesispr_ , zostaliśmy nagrodzeni w kategorii "PR finansowy i relacje inwestorskie" za projekt: INPOST NA EURONEXT AMSTERDAM. Jesteśmy niezmiernie dumni z przyznanego wyróżnienia i idziemy po więcej! 📎 🏆 https://t.co/IY197f3ZZI: 0
@WojciechProma @Piotrski @PaczkomatyPL Tak wygląda międzynarodowa książeczka szczepień, zwana także żółtą książeczką 😜: 0
@Vinzaglio @majakstasko @PaczkomatyPL Mi się trafiły w dodatku taki 20 kg ++ i nikogo w pobliżu. 
Masakra...: -2
@PaczkomatyPL Opcja dla tych co zamawiają chyba  z  aplikacją i podwarunkiem, że będzie miejsce?
Czyli  marketing, który jest sztuką, a nawet więcej...
😉😉😉😉😉: 0
@Piotrski @PaczkomatyPL Nie. Tak wygląda niemiecka książeczka szczepień.: 0
@WojciechProma Zrobili współpracę z @PaczkomatyPL i wypuścili czekoladę o smaku paczkomatu?: -1
@ciynka @majakstasko @PaczkomatyPL @PaczkomatyPL to prawda - juz kilka razy prosiłem o pomoc nieznajomych bo paczka była w najwyżej położonej skrytce włożona + do końca wepchnięta😅: -1
@PaczkomatyPL @ciynka Tyle, że kupując np. na allegro nie mogę wybrać tej opcji. To samo jak nadaję wiele paczek to też ni dy rydy🤷‍♀️🤓: 0
./out/cea3cd56-4e5a-4206-ac3d-be1ddb7148a9.zip is empty! No sentiment for value 755
RT @Polish_Startups: Meet @RBrzoska - Poland's newest billionaire of @PaczkomatyPL fame.
https://t.co/ik963kURLZ: no sentiment
./out/b0f00705-b235-4e31-a93f-5a1018e0e7c1.zip is empty! No sentiment for value 756
Meet @RBrzoska - Poland's newest billionaire of @PaczkomatyPL fame.
https://t.co/ik963kURLZ: no sentiment
@Piechocinski Jak wygląda flota aut elektycznych w @PaczkomatyPL, @InPostPL1 ?: 0
@PaczkomatyPL To czemu wiecznie proszę by ktoś pomógł mi zdjąć paczkę?? Ostatni najwyższy rząd jest dla mnie niedostępny. Ile można podskakiwać??: -1
@Vinzaglio @majakstasko Eeee,  kobiety to nic przy dyskryminacji producentów mebli, budowlańców, sklepów i @PaczkomatyPL...: -1
@MarekZaborek @amazon @PaczkomatyPL Takie zwykle silikonowe za 18pln? Czy jakieś wypasione?: -1
@LeszekJarema @amazon @PaczkomatyPL Etui na AirPods: 0
@MarekZaborek @amazon @PaczkomatyPL A co to za cudo?: 1
@PaczkomatyPL Nie można wysłać pm, wiec o to numer 601459566161493119280332: 0
Podczas #VIKongresPolskiegoKapitału ogłoszono zwycięzców tegorocznych nagród "Forbesa".

Laureatem💥 GRAND PRIX – NAGRODY GŁÓWNEJ PRZEDSIĘBIORCA ROKU MAGAZYNU "FORBES" 💥 został @RBrzoska, twórca InPost @PaczkomatyPL https://t.co/thhbgPK8Jt: 2
Hej @InPostPL1 @PaczkomatyPL kupilem dziecku 30 listopada prezent na mikołajki 6 grudnia, a paczka nie przyszla do dziś, wiec została zgłoszona reklamacja. Dzis się dowiaduje, że potrwa to 30 dni, wiec prezentu tez nie bedzie pod choinkę 24 grudnia? Wesołych Świąt. Tata: 2
Czym jest #InPostFresh? To nowa aplikacja, która pozwala na dokonanie zakupów spożywczych od różnych dostawców. Wykorzystuje dotychczas wypracowane przez #InPost standardy obsługi, technologię mobilną i wysokiej jakości produkty. @PaczkomatyPL @RBrzoska https://t.co/MXT1JfSrtU: 2
InPost… idźcie czasem spać, co? @PaczkomatyPL https://t.co/v5W9gxFFAg: -1
@PaczkomatyPL Czy InPost weryfikuje prawo własności do gruntu pod paczkomat? Chyba nie, skoro postawiliście nielegalnie,bez zezw. Urzędu Miasta - częściowo na działce miejskiej UM Lublin  paczkomat LUB80M przy ul.Platanowej 36.(między 10/2 a 10/4) @prezydentZuk @Miasto_Lublin @PaczkomatyPL https://t.co/acNkPRHZjF: 0
@ktosullivan_uk @PaczkomatyPL @lloyds Hey there! Thanks for getting in touch! I am very sorry you haven't received your order yet! So we can look into this for you please can you drop  us a private message with your order number - Becky: -6
@PaczkomatyPL a rozwozicie w trzech króli paczki czy n?: 0
@PaczkomatyPL dziękuję!: 0
@RealRayWang @GreenlandCorp @PaczkomatyPL would love those for crowd spaces!: 0
@PaczkomatyPL jak mam z dwóch stron naklejki OSTROŻNIE SZKŁO to co to dla was znaczy? bo rozwalić dobrze zabezpieczoną przesyłkę to tez jest sztuka… można jakoś zgłosić przewoźnika? 😡: 1
InPost - @PaczkomatyPL - na mocy zawartej właśnie umowy przez pięć lat będzie obsługiwać dostawy zakupów z platformy ogłoszeniowej OLX

https://t.co/v8SMPxKUzm: 2
@KarenMillen @PaczkomatyPL @lloyds Without doubt the worst online shopping experience I have had.: -2
./out/8b85ed3b-e9fd-4fc9-aca0-2190c25c0ef7.zip is empty! No sentiment for value 776
@KarenMillen @PaczkomatyPL @lloyds https://t.co/sSzs5X3L4Y: no sentiment
RT @theto3k: Przy robieniu zwrotu do paczkomatu naszła mnie taka refleksja, że @PaczkomatyPL to hit co najmniej 20-lecia. Jak by wyglądało…: 0
@emocjewsieci @wirtualnemedia #InPost musi zrobić jakąś kampanię, żeby mniej ludzi się myliło. Wyobraźmy sobie: mówisz InPost, myślisz paczkomaty - @PaczkomatyPL.: -2
@CrazyLukasz Zamawia się do .@PaczkomatyPL 😎: 0
@limaknifares @mmigalski @MrHepik @KONFEDERACJA_ Tylko że regulacja RYNKU przez państwo przeczy wolnemu rynkowi. Państwowa regulacja jest przeciwna wolności.

Dwa największe podmioty ograniczające wolność ludzi to państwo, w którym żyją oraz @PaczkomatyPL.: 0
@WZylm @PaczkomatyPL Panie Wojciechu przykro nam,że ubezpieczyciel zamiast 24,6 PLN wypłacił 20 PLN-niestety powodem było zaznaczenie przez Pana na OLX checkbox Vat-płatnik.zdecydowaliśmy pokryć różnice i przepraszamy za zdarzenie.: 0
@WZylm @PaczkomatyPL @RBrzoska Panie Wojciechu, kwota odszkodowania wynika z faktu, że zaznaczył Pan, iż jest płatnikiem VAT, więc ubezpieczyciel wypłacił kwotę pomniejszoną o podatek. Jednak zdecydowaliśmy zwrócić różnice. Dostanie Pan szczegółową informację w osobnej korespondencji. Wojciech Kądziołka: -2
@PaczkomatyPL jak tam wasza ochrona danych? 🤔 https://t.co/n7fNTCWU4E: 0
RT @theto3k: Przy robieniu zwrotu do paczkomatu naszła mnie taka refleksja, że @PaczkomatyPL to hit co najmniej 20-lecia. Jak by wyglądało…: 0
@WZylm @WKadziolka @PaczkomatyPL @RBrzoska kocham Polskę :) rzecznik prasowy ma zablokowane DMy :D: 0
Uwaga oszuści podszywający się pod @PaczkomatyPL czyhają , zachowujcie ostrożność https://t.co/GFuTNL26Bp: -1
./out/33eb0c46-cf90-41ea-87f1-816f751453ac.zip is empty! No sentiment for value 787
@3sery123 @InPostPL1 @PaczkomatyPL: no sentiment
@podryban @PaczkomatyPL Wesoło. https://t.co/IoYZ3pTAqP: 0
Kolejna ważna, strategiczna umowa #InPost @PaczkomatyPL. Tym razem chodzi o partnerstwo z #OLX. Firma zakłada zanotowanie znacznych wolumenów, dostawy do Paczkomatów są najczęściej wybieraną formą przez klientów #ecommerce w Polsce. @forsalpl @RBrzoska  https://t.co/1DEitPzzn3: 0
@KamilBrzezinski @PaczkomatyPL U nas jest podobnie, tyle, że paczkomat stoi na drobnej skarpie. Pod nią zawsze pełno wody, a jak jest mokro to nie da się wejść po skarpie bez zjechania po błocie prosto do wody... -.-: -6
@WZylm @PaczkomatyPL @RBrzoska Odpisałem w prywatnej wiadomości: 0
@kwiatkowsky1 @WZylm @PaczkomatyPL @RBrzoska No właśnie było nie psuć paczkomatu.: -1
@WZylm @PaczkomatyPL @RBrzoska @Michal__Cwikla karma wrocila 😁😁😁🤣🤣: 0
RT @PaczkomatyPL: Nowy rok, nowy/nowa Ja i nowe Paczkomaty 😎
A gdzie? Między innymi tu:

📦 Podolszyn Nowy,
📦 Chrostkowo,
📦 Żernica,
📦 Korna…: -1
@WKadziolka @PaczkomatyPL @RBrzoska Ma Pan zablokowane DM.: 0
@WZylm @PaczkomatyPL @RBrzoska Moi rodzicom zgubili paczkę o wartości 3 tysięcy (zwrot towaru do sklepu) a wyliczyli straty na 1200 zł 😆: -1
@WZylm @PaczkomatyPL @RBrzoska Panie Wojciechu, proszę na priv nr przesyłki. Zaraz ustalimy o co chodzi
 Wojciech Kądziołka, rzecznik prasowy InPost: 0
@WZylm @PaczkomatyPL @RBrzoska Mi rozwalili paczkę za 150 złotych+koszty przesyłki. Oddali za rozwalony przedmiot, kosztów nie pokryli.: 0
@KamilBrzezinski @PaczkomatyPL Nie oczekiwałem ciągnięcia wątku, moją puentą było jedynie to, że złośliwy typ stał w kupie jeszcze o tym nie wiedząc 😅: -3
@janmazurczak @PaczkomatyPL Nie będę ciągnął tego wątku bo może nas zaprowadzić w dziwne miejsce. Jak widać każdy ma jakieś przygody z Paczkomatami.: 1
@KamilBrzezinski @PaczkomatyPL @PaczkomatyPL tak dbaja o porzadek w miastach :D #greencity by inpost xd: 0
@KamilBrzezinski @PaczkomatyPL Ja używam ALE żeby użyć aplikacji muszę wyciągnąć telefon z kieszeni, a poza domem to nie jest dla mnie takie oczywiste, wolę pokazać QR na zegarku, który myję częściej niż telefon.
Tutaj w sumie spoko by było gdyby Paczkomaty ogarnęły otwieranie skrzynek z Apple Watch 😉: 0
@sztiwen @PaczkomatyPL Dosłownie. Na szczęście w okolicy leżała cegła i można było sobie podstawić: 1
@janmazurczak @PaczkomatyPL Wow. Są jeszcze ludzie, którzy nie używają aplikacji?: 0
@KamilBrzezinski @PaczkomatyPL No to umoczyli: 0
@WZylm @PaczkomatyPL @RBrzoska Gościu wysłał air-tag'a do Korei Północnej i go "zgubili" mimo że wiedział gdzie jest. Pisali do niego z DHL'a z zwrotem kosztów etc bo po prostu mają burdel. Podobnie pewnie u ciebie, szkoda..
https://t.co/sklo3heCn3: -2
@KamilBrzezinski @PaczkomatyPL Ostatnio leżała kupa na wprost komputerka paczkomatu.
Ominąłem ją i kiedy odbierałem paczkę jakiś typ niecierpliwie gapił się na mnie (bo staram się jak najmniej dotykać w takich miejscach, więc chwilę to trwało).
Gapił się na mnie złośliwie stojąc w tej kupie 🤫: -2
./out/baed69f4-4c9b-4c3f-9ce5-3e98cec445c7.zip is empty! No sentiment for value 808
@KamilBrzezinski @PaczkomatyPL Phiii https://t.co/kdEsexsXeh: no sentiment
@BeczekBartek @yeboahkrul @PaczkomatyPL @RBrzoska Dam sobie radę 👍: 0
@WZylm @PaczkomatyPL @RBrzoska a co zamowiles: 0
@WZylm @PaczkomatyPL @RBrzoska dzwon na infolinie i rób burze bo pieniądz rzecz ważna: 1
@WZylm @yeboahkrul @PaczkomatyPL @RBrzoska Załóż zbiórke, ostatnio to modne 👍🏻: 1
@yeboahkrul @PaczkomatyPL @RBrzoska Myśle, że złożyłem reklamację i to jest ich oficjalna decyzja. Pozdrawiam!: 0
@WZylm @PaczkomatyPL @RBrzoska myślę, że od tego jest infolinia a nie twitter. pozdrawiam!: 0
Czasem odbieranie paczek z @PaczkomatyPL to niezła przygoda. Kałuża ma ok 5 cm głębokości na całej długości urządzenia https://t.co/NAuJIvt0lD: 1
@PaczkomatyPL hej, możecie sprawdzić czy wszystko u was ok? Dostałem już 20 maili (w ciagu godziny) z prośba o uzupełnienie formularza reklamacyjnego paczki, której jeszcze nie otrzymałem.: 1
RT @theto3k: Przy robieniu zwrotu do paczkomatu naszła mnie taka refleksja, że @PaczkomatyPL to hit co najmniej 20-lecia. Jak by wyglądało…: 0
@theto3k @PaczkomatyPL Mam tę rozkminę przy każdej paczce. Pamietam czasy wysyłania paczek na Poczta Polska. Dramat [*]: 0
@PaczkomatyPL No to piona! 😉: 0
@Kondzio42 @PaczkomatyPL Niewątpliwie 😆: 0
@theto3k @PaczkomatyPL Stałbyś jeszcze w kolejce po paczki na poczcie z 2021.: 0
Przy robieniu zwrotu do paczkomatu naszła mnie taka refleksja, że @PaczkomatyPL to hit co najmniej 20-lecia. Jak by wyglądało życie bez? 🤷‍♂️: 0
@jagodagrondecka Poproszę. 
Może do @PaczkomatyPL z Afganistanu wysyłają 😜: 0
@PaczkomatyPL Oj jak miło!:) ZGO18M bagno jakich mało w obecnych warunkach pogodowych. W błocie po kolana i dziura na dziurze. Ludzie terenówkami jadą 5km/h. Jest szansa żeby gdzieś zaznaczyć żeby nie wybierano go jako tymczasowy magazyn dla przesyłki?: 1
@michalgdowicz @PaczkomatyPL Kiedyś miałem cztery, każda w innym miejscu😑: 0
Multiskrytka @PaczkomatyPL jest super ale szkoda, że dziś system nie ogarnął, że są dla mnie dwie paczki i jedna wylądowała w paczkomacie docelowym a druga w tymczasowo w innym xD.: 1
Halo @PaczkomatyPL 
stawianie paczkomatów w miejscach gdzie do kosztów paczki trzeba doliczyć koszty naprawy zawieszenia w samochodzie jest nie w porządku:( 

Jeszcze kurier się uwziął na mnie i ciagle wywala mi tam paczki jako do magazynu tymczasowego:(

Jest mi smutno: 0
Tracę wiarę w kurierów. Gdzie kurier InPost zostawił mi paczkę XD
#inpost #paczka #kurier @PaczkomatyPL https://t.co/thrVgNMOzJ: 0
@adam_dworak @PaczkomatyPL InPost to jedna z największych udogodnień w życiu. Szybciutko i sprawnie. Nie stresujesz się, ze kurier wpadnie, jak akurat będziesz szedl na zakupy etc, tylko odbierasz kiedy chcesz. Również uwielbiam.: 1
OLX i @PaczkomatyPL w strategicznym partnerstwie 
https://t.co/dajyxmR7t9: 0
11:20 3.01: Nadanie paczki w Poznaniu
11:40 4.01: Odebranie paczki pod Zamością

Jesteście cudowni, @PaczkomatyPL.: 1
@MadkaKohana Wspaniale wyższa cena za nadal chujową jakość obsługi. Przy konkurenci @PaczkomatyPL poczta żyje tylko dzięki dofinansowaniu z państwa.: 0
@egomont @BlanikZ @PaczkomatyPL Obchodni plany mi neprislusi komentovat my resime technickou cast a provoz plan je urcite pokryti cele CR co nejdrive :-): 0
@marthynovak @BlanikZ @PaczkomatyPL A je v plánu i to, že tím budou moci posílat firmy?: 0
./out/496167d1-bf86-437b-b297-920d44f30bb1.zip is empty! No sentiment for value 835
@egomont @BlanikZ @PaczkomatyPL Urcite brzy :-): no sentiment
@marthynovak @BlanikZ @PaczkomatyPL A kdy to bude v Opavě? Nechci řidiče  PPLky ani vidět...: 0
@ErykMarkowski @PaczkomatyPL O! Dziękuję :): 0
@ErykMarkowski @PaczkomatyPL @gouda89 bardzo fajna: 1
RT @PaczkomatyPL: Nowy rok, nowy/nowa Ja i nowe Paczkomaty 😎
A gdzie? Między innymi tu:

📦 Podolszyn Nowy,
📦 Chrostkowo,
📦 Żernica,
📦 Korna…: -1
./out/cf72003c-f983-4f4b-b9b5-41960ec3b307.zip is empty! No sentiment for value 840
@egomont @BlanikZ @PaczkomatyPL Uz to jde ted: no sentiment
@marthynovak @BlanikZ A půjde přes to i odesílat zásilky, tak jak to děláme přes  @PaczkomatyPL?: 0
Nowy rok, nowy/nowa Ja i nowe Paczkomaty 😎
A gdzie? Między innymi tu:

📦 Podolszyn Nowy,
📦 Chrostkowo,
📦 Żernica,
📦 Kornatka,
📦 Małaszewicze Duże 🔥

Sprawdźcie koniecznie, bo może to właśnie na Was czeka jakaś nowa paczka 😄 https://t.co/1btWogs8hG: 0
@jordanjasinski @KamilDziubka @DHLGlobal @PaczkomatyPL @PocztaPolska Hello and thank you for reaching out to us. We are sorry if you have encountered issues with our service. Please contact us by direct message so that we can assist you to be connected with the relevant unit in your country. Kind regards.: -6
@KamilDziubka U mnie kurier DPD działa świetnie i najlepiej ze wszystkich pozostałych. Jak nie zamawiam przesyłki do paczkomatu @PaczkomatyPL to biorę kuriera DPD.: 0
@KamilDziubka W moim rankingu najgorszej obsługi klienta od lat ścigają się @DHLGlobal w Polsce oraz właśnie @DPDgroup_news. Trudno stwierdzić, która gorsza. Również preferuję @PaczkomatyPL i ostatnio wiele razy pozytywnie zaskoczyła mnie @PocztaPolska.: -5
@mrsGasky @sterlicja Ewentualnie pytanie do @PaczkomatyPL , jak to działa dokładnie: 1
@ktosullivan_uk @PaczkomatyPL @EbuyerHelp Hello there,

I'm glad you told me about this issue. Sorry about this. Please can you drop us a private message and  I'll get this looked into for you. -john: 0
@KarenMillen @PaczkomatyPL Lessons learned, but definitely always buy with credit card. Contrast KM with @EbuyerHelp who when the item was 48 hours late arranged for a replacement to be sent. That is real customer service.: 2
@KarenMillen @PaczkomatyPL So my item was due to delivery on 21st, still no update on where it is or when it would be delivered. No real willingness to resolve the issue in the customer's favour. Had to resort to contacting @lloyds who were excellent. Credited my account and they will recover the cost.: 3
RT @paweljachowski: ‼️ Mój news: #Żabka i #InPost startują ze wspólnym projektem! Paczkomaty typu InDoor staną wewnątrz sklepów największej…: 0
@FanTomasz1976 @niebezpiecznik @PaczkomatyPL Mi dziś dwa smsy przyszły z linkiem do wpłacenia 4,27 zł, bo mam zablokowaną przesyłkę InPostu: 0
Uwaga uwaga @niebezpiecznik @PaczkomatyPL https://t.co/sN17cT8oFV: 0
@SuSanoo_esport @PaczkomatyPL polecam wlasnie mi przyszedl krawat w nich 10 min przed wywiadem: 0
Jak dla mnie piękna sprawa, możesz zawsze odebrać coś od razu, a nie się jebać i łazić gdzieś 
+1 pomyślę nad zamontowaniem u siebie 

@Harpoonlol  X @PaczkomatyPL  options? https://t.co/UF8n1wu7Ce: -3
RT @big_jakub: Rzeszów dostaje ode mnie InPost-owego Oscara. Wczoraj o 15.30 klient nadaje tam paczkę, dziś 11.30 ja, wyciągam ją z paczkom…: -1
RT @paweljachowski: ‼️ Mój news: #Żabka i #InPost startują ze wspólnym projektem! Paczkomaty typu InDoor staną wewnątrz sklepów największej…: 0
‼️ Mój news: #Żabka i #InPost startują ze wspólnym projektem! Paczkomaty typu InDoor staną wewnątrz sklepów największej sieci convenience.
@ZabkaPolska @PaczkomatyPL @RBrzoska 
https://t.co/svdf091BLd: 0
Po raz kolejny mam problem z @PaczkomatyPL. Tym razem odbiór w paczkomacie tymczasowym. Ok nie pierwszy raz. Dzisiaj problem z wyjeciem paczki. Do mniejszej się nie dało włożyć?? https://t.co/ZeKABOEVVx: -1
@PaczkomatyPL To jeszcze po szybkim zerknięciu na mapę w apce: jeżeli jest aktualna, to naprzeciwko WAW502M jest POP-WAW241, a paczka i tak trafia do WAW17M. Na mapie niby blisko, w rzeczywistości jest 6 bliższych paczkomatów. I to inna dzielnica 😂
Przy okazji - WAW17M ma przesunięty dotyk.: 1
@starekonie @PaczkomatyPL Jest masa paczek , paczkomaty są czasami przepełnione i wtedy też zatrzymują tak jakby w jednym miejscu paczkę i czekają aż się zwolni skrytka, zazwyczaj u kuriera który ma wstawić paczkę ale czasem system tego nie aktualizuje na bieżąco :P: 1
@PaczkomatyPL Czy w takim razie jest możliwość wybrania, by w konkretne miejsca paczka nie trafiała? Kiedyś paczkę można było przekierować, teraz jestem stawiany przed faktem dokonanym. 🙄: -1
@PaczkomatyPL - zagadka! Mieszkacie naprzeciwko paczkomatu (wiecznie przepełnionego, ale jednak), paczkopunkt macie równie szczęśliwie 2 ulice dalej a niedaleko na osiedlu są jeszcze 2 inne paczkomaty a ulicę dalej kolejny. Gdzie trafi paczka?: 0
@sstilinska_ @PaczkomatyPL Też chce wiedzieć: 0
RT @sstilinska_: Halo @PaczkomatyPL gdzie moja paczka https://t.co/Ww9hpNc5VK: -1
Rzeszów dostaje ode mnie InPost-owego Oscara. Wczoraj o 15.30 klient nadaje tam paczkę, dziś 11.30 ja, wyciągam ją z paczkomatu w Warszawie. 
Good job @PaczkomatyPL  🏆: -1
@PaczkomatyPL Nic się nie stało, akurat mało ważna przesyłka 😅 dobrego dnia!: 1
Halo @PaczkomatyPL gdzie moja paczka https://t.co/Ww9hpNc5VK: -1
@PaczkomatyPL i #avanspl dokładnie teraz, wspólnie poinformowały się mnie o opóźnieniu w doręczeniu paczki, przepraszają… tylko, ze paczka miała być wczoraj 😂: 2
@MiastoJestNasze @cnkopernik @PaczkomatyPL nigdy w zyciu czegoś takiego nie zacznie używać, bo przecież jazda po chodnikach dla nich to święta rzecz!!: 0
Szybki i niespodziewany #unboxing zanim ruszę na badania 😁
Te zakupy to efekt posiadania @Allegro_Group Smart i darmowej dostawy do @PaczkomatyPL od 40 PLN. Polecam 👍🏻 https://t.co/Q0epPSHMZs: 1
@93ringoXx @PaczkomatyPL ej nie krzycz na InPost, InPost jest super: 1
O proszę. Szacunek @PaczkomatyPL https://t.co/23PGA4vQzk: 0
./out/9ba33b9b-8eac-4ede-a5fa-b46498577a66.zip is empty! No sentiment for value 873
💪😌 https://t.co/XnfitaTYoQ: no sentiment
./out/6b9ae173-55f7-42fb-8ea3-528f3c71c8ab.zip is empty! No sentiment for value 874
@PaczkomatyPL :) 637450599238200111459780: no sentiment
@PaczkomatyPL Nie potrzebuje reklamacji, tylko sprawdzenia co sie stalo z moją paczka i wyslania jej w koncu do mnie: 2
📦 Narok ma nowy Paczkomat i to nie na rok, ale na dłuuugie lata 😅
Mają też inne miejscowości https://t.co/NsBCWxZ4aq.:

📦 Waplewo Wielkie,
📦 Zborowskie,
📦 Stare Strącze,
📦 Oksa 🔥

Kto już miał okazję odbierać lub nadawać w nowej lokalizacji? Łapka do góry 👍 https://t.co/gDIf2TjFSk: 1
@Karczman @PaczkomatyPL mam nadzieje ze nie nie bedzie A, albo paczka nie dotrze uszkodzona, bo są tam rzeczy których nie dorwę w PL "od ręki" :(: -1
Jestem wkurwiona ostro, od 5 dni moja paczka z saf ślęczy w przygotowaniu @PaczkomatyPL MOGLIBYŚCIE ŁASKAWIE RUSZYĆ TYŁEK? #pizgaczhell: -1
@starekonie @PaczkomatyPL Jeżeli wyjdzie opcja A to współczuje kontaktu z infolinia i obsługa reklamacyjną. Przeżyłem na własnej skórze w grudniu.: 0
@starekonie @PaczkomatyPL Dla mnie na serio. Parę razy się na tym przejechałem że paczkomaty wcale nie są 24/7 i paczka w sobotę czy niedzielę nie dotrze. Przestałem kupować/zamawiać rzeczy które nie mogą poczekać do po weekendzie, lub kupuję w poniedziałek.: -2
@CyberKi55052067 @PaczkomatyPL xD to taki bait czy na serio? Jak juz pisalem w odpowiedziach do tego tweeta - paka wyslana z tego samego miejsca zawsze byla w sobote rano BO INPOST NIE MA WOLNEGO W SOBOTE, jest ewidetnie problem z tą przesyłką :D: 0
@starekonie @PaczkomatyPL D. przez sobotę i niedzielę InPost ma wolne i paczka zbiera kurz w magazynie, a Ty uczysz się żeby nie zamawiać rzeczy przed weekendem: 2
@PaczkomatyPL o prosze, plusa wielkiego macie za kontakt tutaj, za chwile wysle pw: 0
@KamilBrzezinski @GraczOjciec @PaczkomatyPL no to chyba jestem szczesciarzem, pierwszy raz widze taka farse u nich: 0
@GraczOjciec @PaczkomatyPL no tez takie patenty kojarze, tylko że tutaj cos totalnie nie gra - zawsze paka wyslana w piatek byla u mnie na paczkomacie w sobote rano, a teraz nikt nic nie wie i na dodatek infolinia pokazała najgorszy możliwy service i babka przez pol minuty powtarzala mi status paczki...: -4
@GraczOjciec @starekonie @PaczkomatyPL Czasem są opóźnienia 2 dni nawet.: 0
@starekonie @PaczkomatyPL Przy weekendzie dane są przekłamane + parokrotnie zdarzało mi się że dostałem info o przygotowaniu paczki, a następne które było w api/na stronie, to info o paczce do odbioru.: 0
Jesli wg apki @PaczkomatyPL paczka zostala wyslana z oddzialu w Krakowie w piatek i od tego czasu nadal jedzie (mamy juz poludnie w poniedzialek) to 
A. paczka zagineła
B. podróż z Krakowa faktycznie trwa ponad 40 godzin
C. system nie pokazuje faktycznego statusu paczki: 0
Pandemia okazała się sporym wyzwaniem dla branży #TSL, która jest jednym z najważniejszych elementów gospodarki, bo łączy transport, spedycję i logistykę. Branża jednak wyszła z kryzysu obronną ręką
@PaczkomatyPL @pkpcargo_media 
https://t.co/W2cg8Rrr8w: 1
@Gotlifebar @Buszkers @DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @_zboral @UD_PragaPolnoc @KubaCzajkowski No i jest. Obok są normalne miejsca parkingowe, zazwyczaj wolne.: 1
@Gotlifebar @Buszkers @DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski O czym kolega pisze? Na każdym zdjęciu widać albo miejsca parkingowe, albo drogę zaraz obok. Jedyne co kurierzy @PaczkomatyPL powinni mieć, to wózeczek na stanie. I tyle.: 2
RT @m3xin: Super paczka w weekend @PaczkomatyPL, przez cały weekend ta paczka ruszyła się o całe 0 km xD

całe szczęście ze to darmowa „eks…: 2
Super paczka w weekend @PaczkomatyPL, przez cały weekend ta paczka ruszyła się o całe 0 km xD

całe szczęście ze to darmowa „ekspresowa” (X D) dostawa z amazon prime, ale macie przestrogę ze to chuja warte jest

nie polecam

#inpost #AmazonPrime: 3
@PaczkomatyPL Hello, kindly assist to confirm if this SMS I received is from inpost. It requests me to pay money for a parcel delivery from AliExpress. The site linked to the sms is to a website that my browser warns me is unprotected. Message in next tweet: 2
@MiastoJestNasze @PaczkomatyPL @RBrzoska @WKadziolka MJN JAKIE RACZYSKO: 0
@Buszkers @DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski No sorry ale paczkomaty są bardziej eko niż normalna kurierka. Wiadomo że nie będzie targał paczek przez ulicę tylko podjeżdża pod paczkomat. 
Problem leży albo w miejscu gdzie jest postawiony Paczkomat albo powinien być poszerzony chodzik, zrobiony jakiś dojazd.: -9
@Andrzej_Cala @PaczkomatyPL A tak naprawdę to zależy od tego kto ma jaką umowę paczki weekendowe mają np posiadacze smarta w allegro ale tak jak mówię to zależy od umowy każdego z inpostem i opcja paczki weekendowej: 1
./out/0dda8b88-52be-43db-845d-88a663f1965e.zip is empty! No sentiment for value 898
@DlaPragi @GazetaStoleczna @jarek_osowski @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski Skorochód-Majewskiego. https://t.co/fqhdx7vEHw: no sentiment
@MiastoJestNasze @PaczkomatyPL @RBrzoska @WKadziolka Czyj jest teren pod paczkomatem? I teren zieleni?: 0
@Andrzej_Cala @PaczkomatyPL To zależy od różnych zmiennych.: 0
@Andrzej_Cala @PaczkomatyPL U mnie czasem pojawiały się w paczkomacie w niedzielę. Więc nie trać nadziei.: 1
@tojazoju @PaczkomatyPL super, dzięki! 🥳: 1
@Andrzej_Cala @PaczkomatyPL Tak.: 0
hej @PaczkomatyPL - przesyłki odebrane od nadawców / zabrane z paczkomatu wczoraj przed godziną 11, trafią dzisiaj do mnie, czy dopiero w poniedziałek?: 0
Francuzi bardziej niż oczekiwano wykorzystują urządzenia przeciwpiechotne @PaczkomatyPL 🙃 https://t.co/NctjjRUKru: 0
RT @MiastoJestNasze: .@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zni…: 0
RT @MiastoJestNasze: .@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zni…: 0
@MiastoJestNasze @PaczkomatyPL @RBrzoska @WKadziolka W październiku trawnik jeszcze nie był rozjechany, a teraz jest jak na zdjęciu wyżej. Ciekawe dlaczego...
@PaczkomatyPL https://t.co/BM0CSuGXCg: 0
RT @MiastoJestNasze: .@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zni…: 0
RT @MiastoJestNasze: .@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zni…: 0
RT @MiastoJestNasze: .@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zni…: 0
@MarekSzolc @jarek_osowski @DlaPragi @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski Proszę bardzo. Zrobiłem je przedwczoraj.: -1
@PaczkomatyPL No ładny wał, jednak historia się powtarza ,, za chwile nie bedzie śladu na giełdzie , inwestorzy …. Bez mydła: 0
@Buszkers @jarek_osowski @DlaPragi @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski Czy mogę wykorzystać zdjęcie? Zwrócę się do miasta, by zinwentaryzowało swoje przestrzenie w okolicach paczkomatów i oficialnie wystąpiło na drodze prawnej o odszkodowania od sprawców tych zniszczeń. To naprawdę skandal i nie wolno nam tego jako Warszawie tolerować.: 0
RT @MiastoJestNasze: .@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zni…: 0
.@PaczkomatyPL @RBrzoska @WKadziolka może jakieś kursy na prawo jazdy dla Waszych kurierów oraz jakaś rekultywacja zniszczonej przez Was zieleni 🌿❓ 

Nie tylko na Grochowskiej, ale w całej Warszawie i innych miastach, gdzie Wasza patologistyka niszczy naszą wspólną przestrzeń‼️ https://t.co/ErgOKtE7zK: -2
./out/d14a2945-4230-4cc3-8b39-f06dd33ebe53.zip is empty! No sentiment for value 917
RT @Buszkers: @DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_Pra…: no sentiment
@bezgotowkowo @PaczkomatyPL @BIPolska Ładny wynik 🥂👍: 0
RT @bezgotowkowo: Z najnowszych danych #InPost wynika, że:
✅ W Polsce działa już ponad 20 tys. @PaczkomatyPL.
✅ W ubiegłym roku liczba dorę…: 1
@adam_dworak @PaczkomatyPL Ciekawie ile razy jeździli po chodniku zeby te paczkę dostarczyć albo ile trawników rozjechali ich dostawcy: -1
@DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski Albo dostawczak na ścieżce rowerowej. Też widok w sumie częsty.: 0
@DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski Paczkomat Grochowska 332 (CC @MiastoJestNasze) https://t.co/Au85Q8BmH2: 0
@DlaPragi @jarek_osowski @GazetaStoleczna @PaczkomatyPL @MiastoJestNasze @JanMencwel @wandagrudzien @daukszu @_zboral @UD_PragaPolnoc @KubaCzajkowski Trzeba nagrywać kierowców i wysyłać na StopAgresjiDrogowej
Jak zaczną dostawać mandaty po 1500zł a firma zmusi kierowców do płacenia, bo to ich wina to wtedy szybko przestanaja jeździć po chodnikach
Ponieważ @PaczkomatyPL ma w dupie skargi na kierowców i nic z tym nie robi: -1
RT @DlaPragi: W dzisiejszej @GazetaStoleczna  @jarek_osowski porusza temat samochodów dostawczych rozjeżdżających trawniki, chodniki i blok…: -1
@PaczkomatyPL Poszło.: 0
@PaczkomatyPL Mogę przez Was na PW?: 0
#InPost i #OLX nawiązały strategiczne partnerstwo w zakresie logistyki przesyłek. Umowę podpisano na 5 lat. Infrastruktura jest w pełni przygotowana na obsługę zwiększonego ruchu - obejmuje już 16 tys. Paczkomatów. @PaczkomatyPL @RBrzoska @wirtualnemedia https://t.co/sLgmE68Qc4: 0
@zakino4 @PaczkomatyPL Przesyłkę jednodniową należy nadać do jakiejś godziny, nie pamietam jakiej. I chyba w dni świąteczne to nie działa. Dlaczego nie widzisz teraz następnego statusu to coś ostatnio złego dzieje się w @PaczkomatyPL, a raczej wśród kurierów tej firmy 🤔: -1
RT @DlaPragi: W dzisiejszej @GazetaStoleczna  @jarek_osowski porusza temat samochodów dostawczych rozjeżdżających trawniki, chodniki i blok…: -1
@zakino4 @PaczkomatyPL Moja nadana w tym samym czasie ze Sznia jest już w Wwa i zaraz trafi do paczkomatu na pradze. Pamiętaj, że był długi weekend…: -1
RT @DlaPragi: W dzisiejszej @GazetaStoleczna  @jarek_osowski porusza temat samochodów dostawczych rozjeżdżających trawniki, chodniki i blok…: -1
Zgadnijcie ile zajmuje przetransportowanie paczki z jednego @PaczkomatyPL do drugiego w tym samym mieście.

Paczka nadana we wtorek o 22. Odebrana przez kuriera o 22:41 tego samego dnia.

Do teraz jej nie dostałem, a nawet nie jest w doręczeniu.

😳: 1
W dzisiejszej @GazetaStoleczna  @jarek_osowski porusza temat samochodów dostawczych rozjeżdżających trawniki, chodniki i blokujących pasy jezdni. Na Pradze też z tym się mierzymy -  @PaczkomatyPL przy Objazdowej - postój na chodniku, wjazd przez przejście... https://t.co/ILkMjmO0Qf: -1
Z najnowszych danych #InPost wynika, że:
✅ W Polsce działa już ponad 20 tys. @PaczkomatyPL.
✅ W ubiegłym roku liczba doręczonych przez firmę przesyłek wyniosła 517,6 mln - a to oznacza wzrost aż o 67% rdr. #WartoBezgotówkowo

Więcej na @BIPolska 👇
https://t.co/jpNT9PPesD: 3
Dobra wiadomość dla wszystkich kupujących na #OLX @PaczkomatyPL podpisał umowę z popularną platformą e-commerce umowę na dostarczanie usług logistycznych. Dzięki temu klienci uzyskają dostęp do ponad 16 tysięcy Paczkomatów w całej Polsce.

https://t.co/uWbQBWMbA6: 1
@OLogistyczny @InPostPL1 Chyba znowu oznaczyliście konto-parodię InPosta. Prawdziwe konto to @PaczkomatyPL. Oznaczacie InPosta, myślcie - paczkomaty. Proste?: 1
@PaczkomatyPL @RBrzoska Czyli mam czekać 30 dni na odbiór paczki? Żart?: 0
@PaczkomatyPL @HERYTlERA A czy w piątek kurierzy dostarczają paczki do domów?: 0
@PaczkomatyPL 2/2 do tego reklamacja przez stronę www IPR11662265. zero reakcji. Przesyłka numer 655080736163400118445212. Mówicie o innowacjach, opakowaniach w ponownym obiegu a nie możecie poradzić sobie z prostą awarią w XXI wieku? @RBrzoska @PaczkomatyPL: -1
@PaczkomatyPL Drodzy, dlaczego nie dbacie o Klientów, którzy mają paczkę w automacie, który uległ awarii. Paczka umieszczona 30.12.21, do dzisiaj nie mogę jej odebrać. Awaria skrytki. 3 telefony co 2 dni na infolinię, która twierdzi, że już dzisiaj na pewno ktoś przełoży paczkę 1/2: -2